# PROJET ANALYSE DE DONNEES
## Etude des stations de location de vélos dans Paris

In [ ]:
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import plotly.express as px
import plotly.express as px
import prince
from prince import CA
%matplotlib inline
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score 
from sklearn.metrics import confusion_matrix


Dans ce notebook, nous allons étudier un jeu de données contenant des informations concernant les taux de disponibilités des vélos dans des stations Vélib parisiennes. Nous avons accès à ce taux pour toutes les heures de chaque jour de la semaine sur la période du 2 septembre au 7 septembre 2014.

Une remarque importante pour comprendre au mieux le sujet est que plus le taux est proche de 1 plus la station a de vélos disponibles. Inversement, quand ce taux est proche de 0 alors il y a de moins en moins de vélos dans la station.

## 1. Présentation des données

Nous allons ici importer toutes les données mises à notre disposition.

In [ ]:
#loading = pd.read_csv('velibLoading.csv', sep = " ")
loading = pd.read_csv('data/velibLoading.csv', sep = " ")
loading

Il y a 168 colonnes qui représentent le total des heures dans une semaine. Les 1189 lignes correspondent aux données de toutes les stations de Paris étudiées.

On affiche ensuite les statistiques descriptives de ce jeu de données. Ici encore les 168 colonnes represéntent chaque heure d'une semaine et les lignes correspondent aux différentes statistiqus calculées à partir du jeu de données précédent.

In [ ]:
loading.describe()

Nous allons maintenant importer les coordonnées géographiques de chaque station. 

In [ ]:
#coord = pd.read_csv('velibCoord.csv', sep = " ")
coord = pd.read_csv('data/velibCoord.csv', sep = " ")

coord.head()

Le jeu de données Coord contient 4 variables: \
1 - position : la latitude et la longitude des 1189 stations \
2 - dates : la date de téléchargement \
3 - bonus : si bonus=1, la station est en altitude et si bonus=0 elle ne l'est pas \
4 - names : le nom des stations. \
Les lignes représentent les stations. 

Nous regardons dans le tableau de données si certaines données sont manquantes.

In [ ]:
print("Nombre de données manquantes dans 'loading':")
print(loading.isna().sum().sum())
print("Nombre de données manquantes dans 'coord':")
print(coord.isna().sum().sum())

Or ici, le résultat obtenu pour chaque tableau "loading" et "coord" est 0, il n'y a donc pas de donnée manquante ! 

Nous affichons ici le nombre d'occurences de chaque station dans nos données, par ordre décroissant.

In [ ]:
print("---Données duppliquées---")
station_name = coord.names.value_counts()
print(station_name)

Le tableau ci-dessous illustre le fait que même si le nom de la station apparaît trois fois dans notre jeu de données, celle-ci ne correspond pas aux mêmes coordonnées géographiques. Cela nous montre bien qu'il ne faut pas supprimer des lignes car même si les stations ont le même nom elles n'ont pas les mêmes caractéristiques géographiques.

In [ ]:
name = station_name.index[0]
coord[coord.names == name]

## 2. Analyse descriptive des données

Afin de poursuivre notre analyse nous allons nous pencher sur le point de vue descriptif.

Pour visualiser un peu mieux ces taux de chargement on affiche aussi le chargement sur la semaine pour 16 stations pris aléatoirement. On peut voir que certaines stations ont des cycles de chargement distinctifs, certaines se chargent la nuit et d'autres le jour, certaines ont un faible taux de chargement toute la semaine, et d'autres ont un profil de chargement distinct entre la semaine et le week-end.

In [ ]:
stations = np.arange(loading.shape[0])
rd.shuffle(stations)
stations = stations[:16] 

# --- #
n_steps    = loading.shape[1] 
fig, axs = plt.subplots(4, 4, figsize = (15,12))
time_range = np.linspace(1, n_steps, n_steps) 


timeTick = np.arange(1, 25*7, 24)

for i in range(4):
    for j in range(4):
        k_station = stations[4 * i + j]
        axs[i, j].plot(time_range, loading.iloc[k_station, :], linewidth = 1, color = 'purple')
        axs[i, j].vlines(x = timeTick, ymin = 0, ymax = 1, colors = "orange", linestyle = "dotted", linewidth = 3)
        axs[i, j].set_title(coord.names[1 + k_station], fontsize = 12)

for ax in axs.flat:
    ax.set_xlabel('Time', fontsize = 12)
    ax.set_ylabel('Loading', fontsize = 12)
    ax.tick_params(axis='x', labelsize=10)
    ax.tick_params(axis='y', labelsize=10)
    
plt.tight_layout()
plt.show()

Observons tout d'abord que les différentes moyennes des stations ne sont pas du tout homogène, et qu'il est donc, intéressant de regarder d'un peu plus près les différentes moyennes observées pour avoir une vision globale de l'impact des heures, jours et de la localisation des stations sur la comportement d'utilisation.

En effet, dans le graphe suivant, il est possible de voir que le taux de chargement moyen varie grandement d'une station à une autre.

In [ ]:
loading_mean = pd.Series(loading.mean(axis=1))
# %load solutions/Python/plot_mean_stations.py
n_stations = loading.shape[0]  # number of observed stations
stations   = np.arange(n_stations)

plt.figure(figsize = (20,6))

# --- #

plt.plot(loading_mean)
plt.hlines(y = loading.mean().mean(), xmin=0, xmax=n_stations, 
           colors = "OrangeRed", linewidth = 3)

# --- #

plt.xlabel('Stations', fontsize = 20)
plt.ylabel('Average loading', fontsize = 20)
plt.title("Average loading per station", fontsize = 25)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)

plt.tight_layout()
plt.show()

Dans un contexte un peu plus général, on affiche la moyenne de chargement totale sur toutes les stations au cours de la semaine.

In [ ]:
print(loading.mean().mean())

Ainsi, on peut dire qu'en moyenne, sur l'ensemble des stations parisiennes et sur une semaine, le taux de chargement moyen est de 0.39. Ce qui signifie qu'en moyenne il y a 4 vélos sur 10 disponibles dans les stations. 

On s'intéresse ici au chargement moyen de toutes les stations en fonction de l'heure (chaque courbe correspond à un jour) et en noire la moyenne sur les jours de ce chargement.

In [ ]:
# On calcule la moyenne pour chaque jour de la semaine
mean_per_hour_per_day = loading.mean(axis = 0).to_numpy()
mean_per_hour_per_day = mean_per_hour_per_day.reshape((7, 24))

mean_per_hour = mean_per_hour_per_day.mean(axis=0)

# --- #

days = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi", "Samedi", "Dimanche"]
plt.figure(figsize = (15,10))

plt.plot(mean_per_hour_per_day.transpose())
plt.plot(mean_per_hour, color = "black", linewidth = 3)

plt.xlabel('Chargement horaire, moyenne de toutes les stations', fontsize = 20)
plt.ylabel('Chargement', fontsize = 20)
plt.legend(days + ['Semaine'])
plt.xticks(ticks = np.arange(0,24,4), labels=np.arange(0,24,4), fontsize = 15)

plt.tight_layout
plt.show()


Ainsi, on peut voir qu'en moyenne, aux alentours de 2h-4h du matin les vélos ne sont pas beaucoup empruntés. Cependant, on peut voir qu'à 9h environ ils le sont davantage, idem entre 19h et 20h. Ce résultat s'explique par les heures de travail de la population qui part le matin en vélo et rentre en fin de journée. 

On souhaite savoir, en moyenne, quel est le jour de la semaine, durant lequel les gens utilisent le plus les vélos, on fait alors une moyenne sur tous les jours de la semaine pour chaque heure de la journée. (Chaque courbe correspond à une heure).

In [ ]:
# On calcule la moyenne pour chaque heure sur tous les jours de la semaine
mean_per_hour_per_day = loading.mean(axis = 0).to_numpy()
mean_per_hour_per_day = mean_per_hour_per_day.reshape((24, 7))

mean_per_hour = mean_per_hour_per_day.mean(axis=0)

# --- #
hours = ["0h", "1h", "2h","3h", "4h", "5h", "6h", "7h", "8h", "9h", "10h", "11h", "12h", "13h", "14h", "15h", "16h", "17h", "18h", "19h", "20h", "21h", "22h", "23h"]
plt.figure(figsize = (20,10))

plt.plot(mean_per_hour_per_day.transpose())
plt.plot(mean_per_hour, color = "black", linewidth = 3)

plt.xlabel('Chargement horaire, moyenne de toutes les stations', fontsize = 15)
plt.ylabel('Chargement', fontsize = 20)
plt.legend(hours + ['hours'])
plt.xticks(ticks = np.arange(0,7,4), labels=np.arange(0,7,4), fontsize = 15)
  
plt.tight_layout
plt.show()

On peut voir que le chargement moyen est quasiment constant chaque jour, même le week-end, contrairement au graphe précédent où le chargement variait légèrement.

On peut donc déjà supposer que l'heure a une plus forte influence sur le profil de chargement que le jour. 


On va maintenant regarder l'évolution du taux sur une station au hasard :

In [ ]:
plt.rcParams['figure.figsize'] = [15, 6]
time_range = np.arange(1, 25)

# on sélectionne aléatoirement un numéro d'un indice de ligne
i = np.random.randint(1, 1190) 

df = loading.iloc[i]

# Création d'une matrice de 24 lignes (une pour chaque heure)
mean_per_hour_per_day = np.array(df).reshape(24, -1)

# on calcul de la moyenne par heure
mean_per_hour = np.mean(mean_per_hour_per_day, axis=1)

# Conversion de la matrice en dataframe
mean_per_hour_per_day = pd.DataFrame(mean_per_hour_per_day)

# Renommage des colonnes avec les jours de la semaine
mean_per_hour_per_day.columns = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi", "Samedi", "Dimanche"]

mean_per_hour_per_day['time_range'] = np.arange(1, 25)
mean_per_hour_per_day = pd.melt(mean_per_hour_per_day, id_vars='time_range', var_name='Days')
mean_per_hour = pd.DataFrame(mean_per_hour)

mean_per_hour.columns = ["Weekly"]

mean_per_hour['time_range'] = np.arange(1, 25)


for day in mean_per_hour_per_day['Days'].unique():
    plt.plot(mean_per_hour_per_day[mean_per_hour_per_day['Days'] == day]['time_range'], 
             mean_per_hour_per_day[mean_per_hour_per_day['Days'] == day]['value'], 
             label=day)

plt.plot(mean_per_hour['time_range'], mean_per_hour['Weekly'], color='black', linewidth=3, label='Weekly')

plt.title(coord.names[i])
plt.xlabel("Heure de la journée")
plt.ylabel("Chargement moyen")
plt.legend()
plt.show()


Regardons maintenant, quelle station est la plus chargée, en moyenne par semaine.

In [ ]:
mean=loading.mean(axis=1)
i = mean.idxmin()
print('Average fill rate :',mean[i])
print(coord.loc[i])

Et de même la station la moins chargée, en moyenne par semaine.

In [ ]:
mean=loading.mean(axis=1)
i = mean.idxmax()
print('Average fill rate :',mean[i])
print(coord.loc[i])

On regarde le comportement moyen par heure (toutes les 4h) sur l'ensemble des stations.

In [ ]:
#on affiche le comportement moyen par heure (toutes les 4h)

hours = [1, 5, 9, 13, 17, 21, 24]

# Taille de la figure et nombre de sous-graphiques
s = 10
n = len(hours)
num_rows = 2
num_cols = (n + num_rows - 1) // num_rows
fig, axs = plt.subplots(num_rows, num_cols, figsize=(s * num_cols, s * num_rows))

for i, h in enumerate(hours):
    load_per_hour = np.mean(loading.iloc[:, h::24], axis=1)  # Calcul de la moyenne pour chaque heure sur tous les jours de la semaine
    row = i // num_cols
    col = i % num_cols
    im = axs[row, col].scatter(coord.latitude, coord.longitude, c=load_per_hour, cmap=cm.plasma_r)
    axs[row, col].set_title('Chargement à {} h'.format(h), fontsize=25)
    plt.colorbar(im, ax=axs[row, col])

for ax in axs.flat:
    ax.set_xlabel('Latitude', fontsize=20)
    ax.set_ylabel('Longitude', fontsize=20)
    ax.tick_params(axis='x', labelsize=15)
    ax.tick_params(axis='y', labelsize=15)

plt.tight_layout()
plt.show()




Ces graphes nous permettent de voir que les zones des stations chargées varient fortement au cours de la journée. On peut supposer un déplacement des vélos vers le centre ville et les bords de Seine le matin entre 8h et 12h (qui pourraient correspondre aux zones de travail) car les stations sont en moyenne plus chargées, et elles semblent se vider entre 16h et 20h, pour se reremplir vers le Sud-Ouest et Sud-Est de Paris, qui pourraient correspondre aux zones d'habitations.

Regardons maintenant le comportement moyen des taux de chargements des stations à 18h pour chaque jour de la semaine. 

In [ ]:
#on affiche le comportement des chargements des stations à 18h chaque jour
loading_data = loading.to_numpy()

h = 18
hours = np.arange(h, 168, 24)

load_per_hour = loading_data[:, hours]

days = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi", "Samedi", "Dimanche"]

# --- #

s, m = 10, 3
k = 1 + len(days)//m

fig = plt.figure(figsize=(s+1, s))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=.3, wspace=.25)

for (i,d) in enumerate(days):
    ax = fig.add_subplot(k, m, i+1)
    im = ax.scatter(coord.latitude, coord.longitude, c = load_per_hour[:,i], cmap = 'viridis')
    plt.colorbar(im)
    
    ax.set_title('Stations de chargement - ' + d + ' {} h'.format(h))
    ax.set_xlabel('Latitude')
    ax.set_ylabel('Longitude')
    ax.tick_params(axis='x')
    ax.tick_params(axis='y')

plt.show()

En comparant le chargement moyen par jour et non plus par heures, on voit que celui-ci semble beaucoup moins varier. L'influence du jour semble moindre.

On fait ici la même chose que précédemment mais on projette les stations sur la carte de Paris, pour avoir une idée plus visuelle de leur localisation.

In [ ]:
Position = np.array(coord)[:,0:2]

#graphe à 1h
a1 = np.arange(1,168,24, dtype=int)
data1h = np.mean(np.array(loading)[:,a1],axis=1)

fig1 = px.scatter_mapbox(data1h, Position[:,1], Position[:,0], color=data1h, color_continuous_scale = px.colors.sequential.Plasma_r, 
                        size_max=15, zoom=10,mapbox_style="carto-positron", opacity = .9,
                        title = 'Chargement à 1h')
fig1.show()

a5 = np.arange(5,168,24, dtype=int)
data5h = np.mean(np.array(loading)[:,a5],axis=1)

fig5 = px.scatter_mapbox(data5h, Position[:,1], Position[:,0], color=data5h, color_continuous_scale = px.colors.sequential.Plasma_r, 
                        size_max=15, zoom=10,mapbox_style="carto-positron", opacity = .9,
                        title = 'Chargement à 5h')
fig5.show()

a9 = np.arange(9,168,24, dtype=int)
data9h = np.mean(np.array(loading)[:,a9],axis=1)

fig9 = px.scatter_mapbox(data9h, Position[:,1], Position[:,0], color=data9h, color_continuous_scale = px.colors.sequential.Plasma_r, 
                        size_max=15, zoom=10, mapbox_style="carto-positron", opacity = .9,
                        title = 'Chargement à 9h')
fig9.show()

a13 =np.arange(13,168,24, dtype=int)
data13h = np.mean(np.array(loading)[:,a13],axis=1)

fig13 = px.scatter_mapbox(data13h, Position[:,1], Position[:,0], color=data13h, color_continuous_scale=px.colors.sequential.Plasma_r, 
                          size_max=15, zoom=10,mapbox_style="carto-positron", 
                         title = 'Chargement à 13h')
fig13.show()

a17 =np.arange(17,168,24, dtype=int)
data17h = np.mean(np.array(loading)[:,a17],axis=1)

fig17 = px.scatter_mapbox(data17h, Position[:,1], Position[:,0], color=data17h, color_continuous_scale=px.colors.sequential.Plasma_r, 
                          size_max=15, zoom=10,mapbox_style="carto-positron",
                         title = 'Chargement à 17h')
fig17.show()


a21 =np.arange(21,168,24, dtype=int)
data21h = np.mean(np.array(loading)[:,a21],axis=1)

fig21= px.scatter_mapbox(data21h, Position[:,1], Position[:,0], color=data21h, color_continuous_scale=px.colors.sequential.Plasma_r, 
                          size_max=15, zoom=10,mapbox_style="carto-positron",
                        title = 'Chargement à 21h')
fig21.show()



a24 =np.arange(24,168,24, dtype=int)
data24h = np.mean(np.array(loading)[:,a24],axis=1)

fig24 = px.scatter_mapbox(data24h, Position[:,1], Position[:,0], color=data24h, color_continuous_scale=px.colors.sequential.Plasma_r, 
                          size_max=15, zoom=10,mapbox_style="carto-positron",
                         title = 'Chargement à 24h')
fig24.show()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from matplotlib import colors
!pip install yellowbrick
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from statsmodels.graphics.mosaicplot import mosaic
from scipy.spatial.distance import cdist

## 3. Etude sur le jeu de données complet 

### 3.1. ACP

Notre étude porte sur un grand jeu de données, nous voudrions trouver un moyen de réduire la dimension de notre jeu de données. Pour ce faire, nous décidons de faire une ACP (Analyse en Composantes Principales) sur notre jeu de donneés afin de réduire un maximum les dimensions.

Pour cela, nous commençons par afficher les boxplots des variables quantitatives sur la même échelle, pour voir s'il est intéressant de standardiser nos données. En effet, si nos données ne sont pas exprimées dans la même échelle/unité cela peut influencer l'importance qu'a une variable sur notre ACP. Ici toutes les données sont comprises entre 0 et 1. Il n'est donc pas nécessaire de standardiser les données. 

In [ ]:
#Affichage en boxplot des variables qualitatives sur la mëme échelle
plt.figure(figsize=(5,5))
loading.boxplot()
plt.title('Boxplot')
plt.xticks(rotation=-90)
plt.xlabel('Colonne')
plt.ylabel('Valeurs')
plt.show()


In [ ]:
pca = PCA()
loading_pca = pca.fit_transform(loading)

In [ ]:
#On affiche la variance expliquée par les composantes de l'ACP
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.title('Variance cumulée expliquée en fonction de la dimension de l espace de l`ACP')
plt.xlabel('Nombre de composantes dans l ACP')
plt.ylabel('Variance cumulée expliquée')

#On regarde le nombre de composantes exacts qu'il nous faut garder afin d'avoir un pourcentage expliqué égal à 80%
pca = PCA(0.80).fit(loading) 

# Affichage du nombre de composantes qu'on garde pour la ACP
pca.n_components_ 
print(f"on garde {pca.n_components_} composants pour le PCA")

La fonction nous dit qu'il faut garder 7 composantes pour expliquer 80% de la variance, cependant cela semble beaucoup , en effet quand on regarde l'histogramme ci-dessous des pourcentages de représentation de chaque composante, on voit qu'après la 5ème composante celles-ci n'explique que très peu la variance.

In [ ]:
#Réalisation de l'ACP à 5 composantes
pca = PCA(n_components=7)

loading_pca = pca.fit_transform(loading) 

print(100*pca.explained_variance_ratio_)

#Affichage et vérification de la dimension du jeu de données avant et après ACP
print('--- PCA ---')
print('Dimension initiale :' , loading.shape)
print('Dimension après projection:', loading_pca.shape)

print('')

print('--- Variance expliquée  ---')
print('Composante 1:', round(pca.explained_variance_[0],2), 'i.e.', round(100*pca.explained_variance_ratio_[0],2), '% de la variance totale')
print('Composante 2:', round(pca.explained_variance_[1],2), 'i.e.', round(100*pca.explained_variance_ratio_[1],2), '% de la variance totale')
print('Composante 3:', round(pca.explained_variance_[2],2), 'i.e.', round(100*pca.explained_variance_ratio_[2],2), '% de la variance totale')


On affiche le barplot représentant la proportion qu'explique chaque variable au niveau de la variance.

In [ ]:
#Affichage du bar plot
plt.figure(figsize=(10, 5))
sns.barplot(x=[f"PC{i+1}" for i in range(len(pca.explained_variance_ratio_))], y=pca.explained_variance_ratio_ * 100)
plt.xlabel("Composantes Principales")
plt.ylabel("Pourcentage de Variance Expliquée")
plt.title("Proportion de Variance Expliquée par Chaque Composante Principale")
plt.ylim(0, 40) 
plt.show()

D'où notre choix de garder seulement 5 composantes.

In [ ]:
#Réalisation de l'ACP à 5 composantes
pca = PCA(n_components=5)
loading_pca = pca.fit_transform(loading) 

Voici l'histogramme et les boxplots représentant le pourcentage de variance expliquée par chaque composante principale:

In [ ]:
#Affichage des bar plot de la proportion d'explication de la variance par chaque composante de l'ACP
plt.figure(figsize=(10, 5))
sns.barplot(x=[f"PC{i+1}" for i in range(len(pca.explained_variance_ratio_))], y=pca.explained_variance_ratio_ * 100)
plt.xlabel("Composantes Principales")
plt.ylabel("Pourcentage de Variance Expliquée")
plt.title("Proportion de Variance Expliquée par chaque composante principale")
plt.ylim(0, 40)  
plt.show()

In [ ]:
#Affichage des boxplots des composantes de l'ACP
box = plt.boxplot(loading_pca[:,:10], patch_artist=True)
plt.setp(box["boxes"], facecolor="coral", alpha=.5)
plt.title("Box plots of the first ten principal components")
plt.tight_layout()
plt.show()

Une fois notre ACP réalisée, on projette nos données dans l'espace d'ACP. 

Voici les graphes de projections des données sur les axes des deux composantes principales de l'ACP, qui expliquent le plus de variance.

In [ ]:
coord1 = pca.components_[0] * np.sqrt(pca.explained_variance_[0])
coord2 = pca.components_[1] * np.sqrt(pca.explained_variance_[1])

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(1, 1, 1)

arrow_length_factor = 3
# Facteur de longueur des flèches

for i, j, nom in zip(coord1, coord2, loading.columns):
    plt.text(i*arrow_length_factor, j*arrow_length_factor, nom, fontsize=10)
    plt.arrow(0, 0, i* arrow_length_factor, j* arrow_length_factor, color = 'purple', alpha=0.7, width = 0.001)

plt.axis((-1, 1, -1, 1))
plt.gcf().gca().add_artist(plt.Circle((0, 0), radius = 1, color = 'cornflowerblue', fill = False))

plt.title('Variables factor map - PCA')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

plt.grid(True)
plt.show()



Et le graphe sur les axes 1 et 3 :

In [ ]:
coord1 = pca.components_[0] * np.sqrt(pca.explained_variance_[0])
coord2 = pca.components_[2] * np.sqrt(pca.explained_variance_[1])

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(1, 1, 1)

arrow_length_factor = 3
# Facteur de longueur des flèches

for i, j, nom in zip(coord1, coord2, loading.columns):
    plt.text(i*arrow_length_factor, j*arrow_length_factor, nom, fontsize=10)
    plt.arrow(0, 0, i*arrow_length_factor, j*arrow_length_factor, color = 'purple', alpha=0.7, width = 0.001)

plt.axis((-1, 1, -1, 1))
plt.gcf().gca().add_artist(plt.Circle((0, 0), radius = 1, color = 'cornflowerblue', fill = False))

plt.title('Variables factor map - PCA')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 3')

plt.grid(True)
plt.show()


Les projections dans les espaces d'ACP peuvent nous amener à faire les hypothèses suivantes : 
- la composante 1 semble correspondre au chargement moyen des stations. 
- Il est compliqué de trouver une interprétation à la composante 2 car la grande quantité de données nous ne permet pas d'avoir un rendu visuel. 

#### Clustering

Dans cette partie nous allons tenter de déterminer des caractéristiques communes à nos données. Grâce à ces points communs, nous allons pouvoir classifier les données et ainsi les diviser en sous-groupes. Cette étape permet d'interpréter au mieux nos données. 

#### 3.1.1. Méthode de clustering avec k-means

L'algorithme K-Means est une technique de classification non-supervisée des données. Il cherche à minimiser les distances entre les observations et les centres des clusters auxquels elles appartiennent.

Cette fonction sera utilisée dans la suite de l'analyse pour faire correspondre les clusters lors de l'analyse sur les données réduits et sur les données complètes pour pouvoir ainsi comparer les résultats.

In [ ]:
def matchClasses(classif1, classif2):
    cm = confusion_matrix(classif1, classif2)
    K = cm.shape[0]
    a, b = np.zeros(K), np.zeros(K)
    for j in range(K):
        for i in range(K):
            if (a[j] < cm[i,j]):
                a[j] = cm[i,j]
                b[j] = i 
    a = a.astype(int)
    b = b.astype(int)
                                             
    print ("")
    print ("Classes size:", a)
    print ("Class (in the classif1 numbering):", b)
    print ("")
    
    table = cm.copy()
    for i in range(K):
        table[:,b[i]] = cm[:,i]   
        
    clusters = classif2.copy()
    n = classif2.shape[0]
    for i in range(n):
        for j in range(K):
            if (classif2[i] == j):
                clusters[i] = b[j]
        
    return table, clusters

Tout d'abord, on va effectuer l'algorithme K-Means sur le jeu de données de base 'loading' et voir si on peut en tirer quelques interprétations.

On va déterminer le nombre de clusters optimal grace à la méthode du coude.

In [ ]:
inertia = []
for k in range(1, 15):
    kmeans = KMeans(n_clusters=k, init='random', n_init='auto', max_iter=100, random_state=42)
    kmeans.fit(loading)
    inertia.append(kmeans.inertia_)
inertia = np.array(inertia)


plt.scatter(range(2, 15), inertia[1:])
plt.xlabel('nombre de clusters', fontsize = 15)
plt.ylabel('inertie', fontsize = 15)
plt.title("Graphe du coude", fontsize = 15)
plt.show()


# --- #


# avec yellowbrick

kmeans = KMeans(init='random', n_init='auto', max_iter=100, random_state=42)
visualizer = KElbowVisualizer(kmeans, k=(1,12))

visualizer.fit(loading)   
visualizer.show()    

Pour analyser ces graphes, on cherche le coude c'est à dire le moment où lorsqu'on va enlève un cluster il y aura une grande augmentation d'inertie, et lorsqu'on en ajoute un, une faible diminution de l'inertie. Dans le premier graphe, on peut observer une différence de tendance entre 2 et le 4. Pour le deuxième graphe, le coude est annoncé à 4. Avec ces graphes on choisirait donc un nombre de clusters de 4.   

Nous allons maintenant utiliser la méthode "silhouette score" pour valider notre choix de clusters

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15,8))

for k in range(2, 6):
    kmeans = KMeans(n_clusters=k, init='random', n_init='auto', max_iter=100, random_state=42)
    q, mod = divmod(k, 2) 
    visualizer = SilhouetteVisualizer(kmeans, colors='yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(loading)

Pour choisir le nombre de clusters le plus adapté il faut trouver un équilibre entre plusieurs aspects. Tout d'abord, il faut que chaque pic dépasse le trait rouge, alors, les données sont adaptées au nombre de classes. Aussi, il faudrait dans l'idéal que chaque pic ait la même épaisseur. Enfin, il faut qu'il y ait le moins de valeurs négatives possibles, elles correspondent au nombre de variables mal classées.

Dans notre cas, le score silouhette pour 4 clusters n'a pas énormément de valeurs négatives et les bandes dépassent le trait. On pourrait aussi choisir de prendre 3 clusters car les résultats sont bons. Mais pour une meilleure interprétation on choisit d'en prendre 4.

In [ ]:
K = 4
kmeans = KMeans(n_clusters=K, init='random', n_init='auto', random_state=0)
#on fixe le nombre de clusters à 4 pour la suite de l'analyse
clusters = kmeans.fit_predict(loading)

Ci-dessous on observe l'histogramme des classifications formées avec le nombre de clusters qui nous avons choisi (ici 4). Cet affichage va nous permettre de voir la répartition des variables dans les clusters et ainsi savoir si certains clusters sont plus importants que d'autres. 

In [ ]:
cmap = plt.get_cmap('Set3',K)
plt.bar(*np.unique(clusters, return_counts=True), color=cmap.colors)
plt.ylabel("Frequency")
plt.show()

On constate que les variables sont réparties de manière assez hétérogène. La majorité des valeurs se trouvent dans le cluster 4 et le reste se trouve assez équitablement réparti dans les clusters 1, 2 et 3.


Avant de tirer des conclusions des analyses qui vont suivre, il faut vérifier que la variance intra-classe de chaque clusters.En effet, cela permet d'évaluer l'homogénéité des groupes et d'être sur que tous les éléments d'un même groupe ont un comportement similaire. Si la variance intraclasse est faible, alors on est assuré de la rigueur de nos analyses. <br> On affiche pour cela la variance intraclasse associée à chaque cluster.

In [ ]:
# distance de chaque point de données à son centre de cluster
distances = kmeans.transform(loading)

#variance intraclasse pour chaque cluster
variances = []
for cluster in range(kmeans.n_clusters):
    cluster_distances = distances[kmeans.labels_ == cluster, cluster]
    variance = np.var(cluster_distances)
    variances.append(variance)

# Affichage
plt.figure(figsize=(8, 6))
plt.bar(range(kmeans.n_clusters), variances, color='skyblue')
plt.xlabel('Clusters')
plt.ylabel('Variance intra-classe')
plt.title('Variance intra-classe par cluster')
plt.xticks(range(kmeans.n_clusters), [f'Cluster {i+1}' for i in range(kmeans.n_clusters)])
plt.show()


On voit que la variance des clusters est assez petite, et de taille équivalente pour chacun des clusters, le profil de chargement des stations dans chaque cluster se ressemble donc. <br>
On affiche maintenant les boxplots qui illustre la répartition des distances de chaque point au centre de son cluster. Nous allons vérifier que cette distribution est dans une fenêtre assez réduite.

In [ ]:
# Calculer les distances de chaque point de données à son centre de cluster
distances = kmeans.transform(loading)

# Créer une liste de distances pour chaque cluster
distances_per_cluster = [[] for _ in range(kmeans.n_clusters)]
for cluster in range(kmeans.n_clusters):
    cluster_distances = distances[kmeans.labels_ == cluster, cluster]
    distances_per_cluster[cluster] = cluster_distances

# Plotter les boxplots
plt.figure(figsize=(10, 6))
plt.boxplot(distances_per_cluster, patch_artist=True, labels=[f'Cluster {i+1}' for i in range(kmeans.n_clusters)])
plt.xlabel('Clusters')
plt.ylabel('Distance au centre de cluster')
plt.title('Distribution des distances au centre de cluster')
plt.show()


Les boxplots associés à chaque clusters ne sont pas très longs, cela est une autre façon d'illustrer les résultats précédents.

On va afficher maintenant le chargement moyen des stations par clusters au cours de la semaine. Cela va peut-être nous permettre de visualiser des tendances communes entre les éléments d'un même cluster.

In [ ]:
loading_sub = loading.iloc[:,:]
loading_sub['cluster']=clusters
mean_loading=loading_sub.groupby('cluster').mean()
mean_loading.head()

col = ['#800080','#008080','#FFFF00','#FF69B4']
fig, ax = plt.subplots(figsize=(10, 6))

# Affichage des 4 lignes sur le même graphique
for i in range(4):
    ax.plot(mean_loading.columns, mean_loading.iloc[i], label=f'Cluster {i+1}', color=col[i])

plt.legend()
plt.title('Moyenne des chargements par cluster')
plt.xlabel('Temps en heures')
plt.ylabel('Moyenne des loading')
plt.show()


Ce graphe représente le chargement moyen des stations en fonction de l'heure et chaque courbe représente un cluster. 
On peut analyser 4 types de comportements différents. 
Un cluster se charge en soirée et se vide le matin avec un chargement qui varie beaucoup.C'est des stations qui pourraient correspondre aux zones d'habitation, où les gens rentrent du travail et chargent les stations et les vident en partant au travail le matin. 

Ensuite, un autre à un comportement inverse au précédent, il correspond aux stations qui se chargent en journée, quand les gens arrivent au travail et se vide quand les gens rentrent chez eux. Le week-end, les variations de chargement de ce cluster sont moins importantes car tous les gens ne se rendent pas dans ces lieux. 

Un autre cluster se charge également en journée mais les stations sont plus vides en moyenne et le chargement varie moins. 

Enfin, à l'inverse du précédent, le dernier cluster se charge en soirée avec des stations plus chargées en moyenne mais avec aussi un chargement qui varie moins. 

On va maintenant tenter de faire un lien entre la variable 'Hill' est les clusters formés par K-Means. Pour ce faire, on dresse une table croisée qui va mettre en lien 'Hill' et les clusters. 

In [ ]:
tbl1 = pd.crosstab(clusters, coord['bonus'])

#on affiche la mosaïque :
tbl1.index = ['Cluster_1', 'Cluster_2', 'Cluster_3', 'Cluster_4'] 
tbl1.columns = ['Pas colline', 'Colline']  

sns.heatmap(tbl1, cmap='coolwarm', annot=True, fmt="d")
plt.title("Stations en altitude en fonction du clusters")
plt.show()

On voit que 119 stations classées dans un cluster sont localisées sur des collines alors que les 8 restantes sont partagées entres les 3 autres clusters. Ce cluster semble être le plus représentatif des stations en altitude, d'après le graphe des moyennes, il représente aussi les stations aux taux souvent bas voir très bas, ce qui signifie qu'il n'y a pas beaucoup de vélo dans la station, les vélos sont donc empruntés mais jamais ramenés.

In [ ]:
# Etude sur loading_pca

Maintenant, on va effectuer l'algorithme K-Means à nouveau mais cette fois-ci, sur le jeu de données réduit par l'ACP. Cela va permettre de voir si les données réduites représentent bien le jeu de données complet. On cherche entre autre à voir si la classification réalisée sur le jeu de données complet et la même sur celui réduit, si c'est le cas cela nous permettra de simplifier les calculs des algorithmes suivants en travaillant sur le jeu de données réduit.

On va déterminer le nombre de clusters optimal grace à la méthode du coude.

In [ ]:
inertia = []
for k in range(1, 15):
    kmeans_pca = KMeans(n_clusters=k, init='random', n_init='auto', max_iter=100, random_state=42)
    kmeans_pca.fit(loading_pca)
    inertia.append(kmeans_pca.inertia_)
inertia = np.array(inertia)


plt.scatter(range(2, 15), inertia[1:])
plt.xlabel('nombre de clusters', fontsize = 15)
plt.ylabel('inertie', fontsize = 15)
plt.title("Graphe du coude", fontsize = 15)
plt.show()


# --- #


# Using yellowbrick

kmeans_pca = KMeans(init='random', n_init='auto', max_iter=100, random_state=42)
visualizer = KElbowVisualizer(kmeans_pca, k=(1,12))

visualizer.fit(loading_pca)    # Fit the data to the visualizer
visualizer.show()    # Finalize and render the figure

D'après la méthode du coude on peut dire que le nombre de clusters le plus adapté avec K-means est de 4. 

Nous allons utiliser la méthode "silouhette score" pour valider le résultat précedent.

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15,8))

for k in range(2, 6):
    kmeans_pca = KMeans(n_clusters=k, init='random', n_init='auto', max_iter=100, random_state=42)
    q, mod = divmod(k, 2) 
    visualizer = SilhouetteVisualizer(kmeans_pca, colors='yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(loading_pca)

On peut voir que le graphe pour 4 clusters est très correct : peu de valeurs négatives et des clusters qui dépassent tous la moyenne. On aurait aussi pu choisir 3.

In [ ]:
K = 4
kmeans_pca = KMeans(n_clusters=K, init='random', n_init='auto', random_state=0)
clusters_pca = kmeans_pca.fit_predict(loading_pca)
#On fixe le nombre de clusters à 4

Regardons maintenant la répartition des données au sein des clusters. Si on choisit un nombre de clusters égal à 4, voici l'histogramme des données associé :

In [ ]:
cmap = plt.get_cmap('Set3',K)
plt.bar(*np.unique(clusters, return_counts=True), color=cmap.colors)
plt.ylabel("Frequency")
plt.show()

La répartition des valeurs est équivalente à celle pour l'étude de loading complet.

On visualise ici la répartition des valeurs dans le plan de l'ACP (dimention 1 et 2) pour voir comment se comporte les clusters.

In [ ]:
import matplotlib.pyplot as plt

# Définition des couleurs des clusters
colors = ['#800080','#008080','#FFFF00','#2E4E7E']

plt.figure(figsize=(8, 6))
# Tracé du nuage de points des clusters dans l'espace d'ACP
scatter = plt.scatter(loading_pca[:, 0], loading_pca[:, 1], c=clusters_pca, cmap='viridis', s=50, alpha=0.5)

# Définition des étiquettes de légende pour chaque cluster
legend_labels = ['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4']

# Création des marqueurs de légende avec les couleurs correspondantes
legend_markers = [plt.Line2D([0], [0], linestyle='none', marker='o', color=color, markersize=10) for color in colors]

# Affichage de la légende avec les marqueurs et les étiquettes définis
plt.legend(legend_markers, legend_labels, loc='lower right')

plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Clusters dans le plan de l\'ACP')
plt.show()


Les variables en dim1 et dim2 sont bien réparties et les clusters ne se chevauchent pas.

Avant de tirer des conclusions des analyses qui vont suivre, il faut vérifier que la variance intra-classe de chaque clusters.

In [ ]:
# distance de chaque point de données à son centre de cluster
distances = kmeans_pca.transform(loading_pca)

#variance intraclasse pour chaque cluster
variances = []
for cluster in range(kmeans_pca.n_clusters):
    cluster_distances = distances[kmeans_pca.labels_ == cluster, cluster]
    variance = np.var(cluster_distances)
    variances.append(variance)

# Affichage
plt.figure(figsize=(8, 6))
plt.bar(range(kmeans_pca.n_clusters), variances, color='skyblue')
plt.xlabel('Clusters')
plt.ylabel('Variance intra-classe')
plt.title('Variance intra-classe par cluster')
plt.xticks(range(kmeans_pca.n_clusters), [f'Cluster {i+1}' for i in range(kmeans_pca.n_clusters)])
plt.show()


Les variances sont assez faibles donc on peut faire des analyses sur ces données

On va maintenant tenter de faire un lien entre la variable 'Hill' est les clusters formés par K-Means. Pour ce faire, on dresse une table croisée qui va mettre en lien 'Hill' et les clusters. 

In [ ]:
tbl1_2 = pd.crosstab(clusters_pca, coord['bonus'])

#on affiche la mosaïque :
tbl1_2.index = ['Cluster_1', 'Cluster_2', 'Cluster_3', 'Cluster_4'] 
tbl1_2.columns = ['Pas colline', 'Colline']  

sns.heatmap(tbl1_2, cmap='coolwarm', annot=True, fmt="d")
plt.title("Stations en altitude en fonction du clusters")
plt.show()

Ici, on voit que les stations en altitude sont globalement regroupées dans un cluster. En effet, on remarque que dans la ligne 1 (les stations en altitudes), 118 variables sont dans le même cluster alors que les 8 autres variables sont divisées entre les clusters. Ainsi, ce cluster de cette classification ressemble au cluster de la classification sur le jeu de données complet où les stations en altitude étaient le plus présente.

In [ ]:
loading_sub2 = loading.iloc[:,:]
loading_sub2['cluster']=clusters_pca
mean_loading2=loading_sub2.groupby('cluster').mean()
mean_loading2.head()

col = ['#800080','#008080','#FFFF00', '#FF69B4']
fig, ax = plt.subplots(figsize=(10, 6))

# Affichage des 4 lignes sur le même graphique
for i in range(4):
    ax.plot(mean_loading2.columns, mean_loading2.iloc[i], label=f'Cluster {i+1}', color=col[i])

plt.legend()
plt.title('Moyenne des chargements par cluster')
plt.xlabel('Temps en heures')
plt.ylabel('Moyenne des loading')
plt.show()


Un cluster se charge en soirée et se vide le matin avec un chargement qui varie beaucoup.
Ensuite, un autre à un comportement inverse au précédent, il correspond aux stations qui se chargent en journée, et se vide quand les gens rentrent chez eux. Le week-end, les variations de chargement de ce cluster sont moins importantes. 

Un autre cluster se charge également en journée mais les stations sont plus vides en moyenne et le chargement varie moins. 

Enfin, à l'inverse du précédent, le dernier cluster se charge en soirée avec des stations plus chargées en moyenne mais avec aussi un chargement qui varie moins. 

Finalement, les profils de chargement des clusters se ressemblent fortement entre les données pca et non transformées.

In [ ]:
# Comparaison des classification avec scatterplot

Comparaison des deux classifications grâce à un scatterplot pour voir le nombre de valeurs qui correspondent dans le cas normal et le cas réduit, afin de voir si il est vraiment utile de faire son analyse sur l'ACP ou non ? :

In [ ]:
Position = np.array(coord)[:,0:2]

#nuage de points pour loading entier 
f1 = px.scatter_mapbox(clusters, Position[:,1], Position[:,0], color=clusters, color_continuous_scale=[(0, '#800080'), (0.5, '#008080'), (1, '#FFFF00')], 
                        size_max=15, zoom=10,mapbox_style="carto-positron", opacity = .9,
                        title = 'clusters données complètes')

f1.show()

f2 = px.scatter_mapbox(clusters_pca, Position[:,1], Position[:,0], color=clusters_pca, color_continuous_scale =[(0, '#800080'), (0.5, '#008080'), (1, '#FFFF00')], 
                        size_max=15, zoom=10,mapbox_style="carto-positron", opacity = .9,
                        title = 'clusters données pca')
f2.show()



#reclassement des clusters pour matcher les couleurs
cm1, clusters_pca_sorted = matchClasses(clusters, clusters_pca)

#quels sont les pints identiques ?
points_diff = clusters != clusters_pca_sorted

num_diff_points = np.sum(points_diff)

pourcentage_reussite = (1- num_diff_points / len(clusters)) * 100

print(f"Nombre de points différents : {num_diff_points} sur {len(clusters)}")
print(f"Pourcentage de réussite : {pourcentage_reussite:.2f}%")




Le pourcentage de réussité est vraiment très important, seulement 6 points ne correspondent pas, les données matchent bien. Vérifions cela grâce à la matrice de confusion.

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(clusters, clusters_pca)).plot()
plt.xlabel('Sur les données réduites (PCA)')
plt.ylabel('Sur les données complètes')
plt.show()

In [ ]:
#on reclasse nos clusters

In [ ]:
ConfusionMatrixDisplay(cm1).plot()
plt.xlabel('On the reduced data (PCA)')
plt.ylabel('On the complete data')
plt.show()

Ici aussi on remarque que les données sont très bien classées, la matrice est quasiment diagonale.

Pour la suite des analyses, on va pouvoir les effectuer sur loading_pca, le jeu de données réduit car il représente très bien le jeu de données complet et permet de limiter les temps de calculs.

In [ ]:
#on convertit la table de confusion en dataframe 
df_cm =pd.DataFrame(cm1, columns=['groupe0', 'groupe1', 'groupe2','groupe3'], index=['groupe_pca0', 'groupe_pca1', 'groupe_pca2','groupe_pca3'])
df_cm.head()

En affichant la table de contingence, on voit que le nombre de stations mal classifié est très faible, cela nous permet de conclure (directement et non en passant par une CA) que nos méthodes de classification sont comparables. Et donc que nos méthodes de classification sur les données d'ACP réprésentent très bien la situation avec les données complètes. 

#### 3.1.2. CAH : Agglomerative Clustering


Etudions ici la méthode de la classification ascendante hiérarchique (CAH). Cette méthode consiste à afficher des dendrogrammes des différente données en fonction de différents likages.

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

La méthode CAH étant très coûteuse en termes de calculs, on préférera faire l'analyse en prenant un échantillon plus petit. Nous avons vu que l'analyse des données ACP donnait de bons résultats, donc pour faciliter les calculs, lors des analyses suivantes nous prendrons les données réduites. 

In [ ]:
print("Dimension du jeu de données entier 'loading' :" , loading.shape)
print("Dimension du jeu de données réduit 'loading_pca': " +str(loading_pca.shape))

On voit bien que la dimension est grandement réduite.

On détermine le nombre de clusters adéquats grâce aux critères graphiques, notamment le "graphe du coude".

In [ ]:
#Réalisation du CAH avec le linkage 'Ward' 
ac = AgglomerativeClustering(linkage="ward", compute_distances=True)
clusterscah1= ac.fit_predict(loading_pca)

#Affichage du nombre de clusters à garder en fonction du critère du coude
ac = AgglomerativeClustering(linkage='ward', compute_distances=True)
visualizer = KElbowVisualizer(ac, k=(1,12))

visualizer.fit(loading_pca)  # Fit the data to the visualizer
visualizer.show()   
plt.show()

D'après les résultats graphiques obtenus en R et en Python, nous décidons de prendre 4 clusters différents. Cela nous permet également d'être en accord avec le nombre de clusters de Kmeans.

On affiche alors le dendrogramme associé en utilisant le linkage "Ward".

In [ ]:
#Nombre de clusters choisi par les indicateurs 
K = 4

#Réalisation de la CAH avec le linkage 'Ward' pour K=4 clusters
ac = AgglomerativeClustering(n_clusters=K, compute_distances=True, linkage='ward')
clusterscah1 = ac.fit(loading_pca)
print(clusterscah1)

children = ac.children_
distances = ac.distances_
n_observations = np.arange(2, children.shape[0]+2)
linkage_matrix = np.c_[children, distances, n_observations]

sch.dendrogram(linkage_matrix, labels=ac.labels_)

# On coupe le dendrogramme à K=4 clusters
max_d = .5*(ac.distances_[-K]+ac.distances_[-K+1])
plt.axhline(y=max_d, c='k')

plt.title("Dendrogramme avec le linkage Ward")
plt.show()

On affiche tous les types de linkages que l'on a étudié pour voir les différences sur un même jeu de données.

In [ ]:
#Affichage des dendrogrammes avec les différents linkages afin de pouvoir voir les différences sur un même jeu de données

plt.subplot(2,2,1)
linkage_matrix_single = sch.linkage(loading_pca, method='single')
sch.dendrogram(linkage_matrix_single)
plt.title("Dendrogramme avec le linkage simple")

plt.subplot(2,2,2)
linkage_matrix_complete = sch.linkage(loading_pca, method='complete')
sch.dendrogram(linkage_matrix_complete)
plt.title("Dendrogramme avec le linkage complete")

plt.subplot(2,2,3)
linkage_matrix_average = sch.linkage(loading_pca, method='average')
sch.dendrogram(linkage_matrix_average)
plt.title("Dendrogramme avec le linkage moyenne")

plt.subplot(2,2,4)
linkage_matrix_ward = sch.linkage(loading_pca, method='ward')
sch.dendrogram(linkage_matrix_ward)
plt.title("Dendrogramme avec le linkage Ward")

plt.tight_layout()
plt.show()

Ainsi, les affichages nous montre bien que le likage "Ward" nous permet d'avoir des clusters de taille à peu près équivalentes (voir code R) et il nous permet de représenter des chargements moyens cohérents. De plus, on voit bien graphiquement que 4 clusters correspond bien au saut le plus important dans le graphe de 'Ward', la mesure de dissimilarité entre les clusters est élevé. 


De plus, la suite des interprétations sur R nous donne quasiment les mêmes informations que les KMeans, avec un cluster faiblement chargée en moyenne et qui varie peu qui contient la majorité des stations en altitude, et des clusters de jours et de nuits.

#### 3.1.3. Gaussian Mixture Models

Afin de mettre en place la méthode du GMM, on applique un critère de sélection du nombre de clusters. On choisit ici de travailler avec le crière BIC.

In [ ]:
# Critère BIC

In [ ]:
k_max = 15

bic = []
for k in range(2, k_max):
    gmm = GaussianMixture(n_components=k, init_params='kmeans', n_init=3)
    gmm.fit(loading_pca)
    bic.append(gmm.bic(loading_pca))
bic = np.array(bic)

plt.scatter(range(2, k_max), bic)
plt.show()

On cherche ici à obtenir le score BIC le plus petit possible, ainsi on voit que le modèle expliquant au mieux nos données contient 6 clusters.

Nous décidons de faire notre étude avec 4 clusters en Python car ceci correspond aussi au nombre de clusters pris pour Kmeans, on à pas une grande différence de score BIC entre 4 et 6 clusters et enfin, avoir 6 clusters serait compliqué à interpréter.

On affiche d'abord la dispersion des points par cluster initiée par le modèle de mélange gaussien (GMM)

In [ ]:
K = 4
cmap = plt.get_cmap('Set3', K)

gmm = GaussianMixture(n_components=K, n_init=3)
clusters_gmm_loading = gmm.fit_predict(loading_pca)

# --- #

plt.subplot(1,2,1)
scatter = plt.scatter(loading_pca[:,0], loading_pca[:,1], c=clusters_gmm_loading, s=1, linewidths=1, cmap='viridis')

legend_labels = [f'Cluster {i+1}' for i in range(4)]
plt.legend(handles=scatter.legend_elements()[0], labels=legend_labels)
plt.grid(True)


plt.tight_layout()
plt.show()

Avant de continuer nos analyses
, on va vérifier que la variance dans chaque cluster ne soit pas trop grande, ainsi les élèments d'un même cluster se comporteraient en moyenne de la même manière. <br> On affiche pour cela les boxplots des données de chaque cluster.

In [ ]:
aux = pd.DataFrame({
    'label': ['Cluster ' + str(label) for label in clusters_gmm_loading],
    'proba': np.max(gmm.predict_proba(loading_pca), axis=1)
})
sns.boxplot(x='label', y='proba', data=aux, palette='Set3', showfliers=False )
plt.title("Probabilité d'appartenance au cluster")
plt.show()


On peut voir que la plupart des stations semblent bien classées car leur probabilité d'appartenance au cluster est proche de 1.

On affiche désormais le plot mosaïque, qui permet de voir combien de stations de chaque cluster sont en altitude ou non:

In [ ]:
tbl1_1 = pd.crosstab(clusters_gmm_loading, coord['bonus'])

#on affiche la mosaïque :
tbl1_1.index = ['Cluster_1', 'Cluster_2', 'Cluster_3', 'Cluster_4'] 
tbl1_1.columns = ['Pas colline', 'Colline']  

sns.heatmap(tbl1_1, cmap='coolwarm', annot=True, fmt="d")
plt.title("Stations en altitude en fonction du clusters")
plt.show()


On peut voir qu'un cluster comporte plus de stations en altitude que les autres en proportion.

On affiche maintenant le chargement moyen des stations par clusters (issus de GMM) au cours de la semaine. Cela va peut-être nous permettre de visualiser des tendances communes entre les éléments d'un même cluster.

In [ ]:
loading_sub5 = loading.iloc[:,:]
loading_sub5['cluster']=clusters_gmm_loading
mean_loading5=loading_sub5.groupby('cluster').mean()
mean_loading5.head()


fig, ax = plt.subplots(figsize=(10, 6))
col= ['purple', 'blue'  ,'green','yellow']

# Affichage des 4 lignes sur le même graphique

for i in range(4):
    ax.plot(mean_loading5.columns, mean_loading5.iloc[i], label=f'Cluster {i+1}',color=col[i])

    
plt.legend()
plt.title('Moyenne des chargements par cluster')
plt.xlabel('Temps en heures')
plt.ylabel('Moyenne des loading')
plt.show()

On retrouve certains profils de clusters classiques : un cluster qui se charge en journée, qui pourrait correspondre aux zones de travails, et les autres clusters ont un profil de chargement plus nocturne. Le cluster avec un faible chargement en moyenne contient encore la majorité des stations en altitude. 

L'avantage d'avoir gardé 4 clusters en Python nous permet de faire une étude comparative des méthodes de classification comme ce qui suit : 

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(clusters_gmm_loading, clusters_pca)).plot()
plt.xlabel('On the reduced data (PCA)')
plt.ylabel('On the complete data')
plt.show()

In [ ]:
cm1_GMM, clusters_pca_sorted_jour_GMM = matchClasses(clusters_gmm_loading, clusters_pca)

#Reclassement des groupes 
df_Kmeans_gmm1 = pd.DataFrame(cm1_GMM, columns=['groupe0', 'groupe1', 'groupe2','groupe3'], index=['groupe_pca0', 'groupe_pca1', 'groupe_pca2','groupe_pca3'])
df_Kmeans_gmm1.head()

En affichant la table de contingence, on voit que le nombre de stations mal classifié est important, on va réaliser une Analyse en Correspondance (CA) pour étudier la situation :

In [ ]:
#Réalisation de la CA 
ca = prince.CA(
     n_components=14,
     n_iter=10,
     copy=True,
     check_input=True,
     engine='sklearn',
     random_state=42)

ca = ca.fit(df_Kmeans_gmm1)

In [ ]:
#Affichage des valeurs propres 
display(ca.eigenvalues_summary)

ca.scree_plot()

In [ ]:
gmm_coord = ca.column_coordinates(df_Kmeans_gmm1)
gpe_pca_coord = ca.row_coordinates(df_Kmeans_gmm1)

display(gmm_coord)
display(gpe_pca_coord)

In [ ]:
#Affichage des points 
plt.scatter(gmm_coord.iloc[:, 0], gmm_coord.iloc[:,1], color='blue', label='clusters_GMM') 
plt.scatter(gpe_pca_coord.iloc[:, 0], gpe_pca_coord.iloc[:, 1], color='green', label='Clusters_PCA') 


plt.legend()
plt.show()

Ce graphe permet de projeter les points dans l'espace d'ACP, sur les composantes 0 et 1 qui expliquent à elles deux plus de 80% de la variance.

On voit globalement que le cluster_gmm_i est représenté par le clusters_pca_i quelque soit i entre [0,3].

En conclusion, on voit que les clusters issus des deux méthodes sont assez similaires pour l'ensemble des clusters et on peut donc conclure que les résultats de notre CA sont satisfaisants. Autrement dit nos deux méthodes de classifications classent de la même façon nos données. 

### 3.2. CA : comparaison entre Kmeans et Colline

On décide désormais d'étudier s'il existe un lien entre la classification opérée sur loading_pca par Kmeans et les stations localisées sur une colline.

Pour ce faire, on réalise une Analyse en Correspondance. On commence par calculer la matrice de confusion :

In [ ]:
cm = confusion_matrix(clusters_pca, coord['bonus'])
print(cm[:,0:2])
cm1 = cm[:,0:2]
ConfusionMatrixDisplay(cm).plot()
plt.xlabel('Groupes Hill 0 et 1')
plt.ylabel('Clusters_pca issus de Kmeans')
plt.show()

In [ ]:
#on convertit la table de confusion en dataframe 
df_cm = pd.DataFrame(cm1 , columns=['Pas colline', 'Colline'], index=['groupe_pca0', 'groupe_pca1', 'groupe_pca2','groupe_pca3'])
df_cm.head()

On utilise la bibliothèque CA du package "prince".

In [ ]:
#Réalisation de la CA 
ca = prince.CA(
     n_components=14,
     n_iter=10,
     copy=True,
     check_input=True,
     engine='sklearn',
     random_state=42)

ca = ca.fit(df_cm)

In [ ]:
#Affichage des valeurs propres 
display(ca.eigenvalues_summary)

ca.scree_plot()

In [ ]:
hill_coord = ca.column_coordinates(df_cm)
gpe_coord = ca.row_coordinates(df_cm)

display(hill_coord)
display(gpe_coord)

In [ ]:
#Affichage des points 
plt.scatter(hill_coord, [0, 0], color='blue', label='Hill')
plt.scatter(gpe_coord, [0, 0, 0,0], color='green', label='Clusters')

plt.legend()
plt.show()


Ce graphe permet de projeter les points dans l'espace d'ACP. On voit que 3 clusters ainsi que les stations qui ne sont pas en altitude sont regroupées ensemble. Les stations en altitude sont éloignées de tous les clusters mais restent plus proche d'un cluster, elle vont donc majoritairement se situées dans celui-ci.

In [ ]:
ca.column_cosine_similarities(df_cm)

Cela nous permet de voir que les variables 'colline' - 'pas colline' très bien représentées.

In [ ]:
ca.row_cosine_similarities(df_cm)

Cela nous permet de voir que les différents groupes de clusters formés sur les données pca sont très bien représentés.

## 4. Etude sur le jeu de données par jour

Nous avons décidé de créer un nouveau jeu de données moyenné sur les jours afin d'espérer trouver de nouvelles déductions de nos données. Autrement dit, on crée un jeu de données qui comporte 7 vairables (pour les 7 jours de la semaine). 

### 4.1. ACP

On commence par regarder s'il est possible de réduire encore une fois la dimension de notre nouveau dataframe et de potentiellement en déduire des liens entre les taux de chargement et les jours. 

Pour ce faire, nous décidons de faire une ACP sur notre jeu de donneés.

Ici encore la question de la standardisation des données se posent, or les données sont toujours comprises entre 0 et 1. Il n'est donc pas nécessaire de standardiser les données. 

In [ ]:
lundi = loading.iloc[:, list(range(0, 24))].mean(axis=1)
mardi = loading.iloc[:, list(range(24, 48))].mean(axis=1)
mercredi = loading.iloc[:, list(range(48, 72))].mean(axis=1)
jeudi = loading.iloc[:, list(range(72, 96))].mean(axis=1)
vendredi = loading.iloc[:, list(range(96, 120))].mean(axis=1)
samedi = loading.iloc[:, list(range(120, 144))].mean(axis=1)
dimanche = loading.iloc[:, list(range(144, 168))].mean(axis=1)

data_jours = pd.DataFrame({
    'lundi': lundi,
    'mardi': mardi,
    'mercredi': mercredi,
    'jeudi': jeudi,
    'vendredi': vendredi,
    'samedi': samedi,
    'dimanche': dimanche
})

data_jours['Hill'] = coord['bonus']


print(data_jours)

In [ ]:
pca_2 = PCA()
data_jours_pca = pca_2.fit_transform(data_jours)

In [ ]:
#On affiche la variance expliquée par les composantes de l'ACP
plt.plot(np.cumsum(pca_2.explained_variance_ratio_))

plt.title('Variance cumulée expliquée en fonction de la dimension de l espace de l`ACP')
plt.xlabel('Nombre de composantes dans l ACP')
plt.ylabel('Variance cumulée expliquée')

#On regarde le nombre de composantes exacts qu'il nous faut garder afin d'avoir un pourcentage expliqué égal à 85%
pca_2 = PCA(0.85).fit(data_jours) 

#Nombre de composantes gardées pour l'ACP
pca_2.n_components_ 
print(f"on garde {pca_2.n_components_} composantes pour le PCA")

La fonction nous dit qu'il faut garder 3 composantes pour expliquer 85% de la variance.

In [ ]:
#Réalisation de l'ACP avec 2 composantes
pca_2=PCA(n_components=2)
data_jours_pca = pca_2.fit_transform(data_jours) 
print(100*pca_2.explained_variance_ratio_)

#Affichage et vérification de la dimension du jeu de données avant et après ACP
print('--- PCA ---')
print('Dimension initiale :' , data_jours.shape)
print('Dimension après projection:', data_jours_pca.shape)

print('')

print('--- Variance expliquée  ---')
print('Composante 1:', round(pca.explained_variance_[0],2), 'i.e.', round(100*pca.explained_variance_ratio_[0],2), '% de la variance totale')
print('Composante 2:', round(pca.explained_variance_[1],2), 'i.e.', round(100*pca.explained_variance_ratio_[1],2), '% de la variance totale')

In [ ]:
#Affichage des bar plot de la proportion d'explication de la variance par chaque composante de l'ACP
plt.figure(figsize=(10, 5))
sns.barplot(x=[f"PC{i+1}" for i in range(len(pca_2.explained_variance_ratio_))], y=pca_2.explained_variance_ratio_ * 100)
plt.xlabel("Composantes Principales")
plt.ylabel("Pourcentage de Variance Expliquée")
plt.title("Proportion de Variance Expliquée par Chaque Composante Principale")
plt.ylim(0, 80) 
plt.show()

In [ ]:
#Affichage des boxplots des composantes de l'ACP
box = plt.boxplot(data_jours_pca[:,:10], patch_artist=True)
plt.setp(box["boxes"], facecolor="coral", alpha=.5)
plt.title("Box plots des 2 premières composantes")
plt.tight_layout()
plt.show()

On projette nos données dans sur les axes de l'ACP :

In [ ]:
coord1 = pca.components_[0] * np.sqrt(pca_2.explained_variance_[0])
coord2 = pca.components_[1] * np.sqrt(pca_2.explained_variance_[1])

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(1, 1, 1)


arrow_length_factor = 5

for i, j, nom in zip(coord1, coord2, data_jours.columns):
    plt.text(i*arrow_length_factor, j*arrow_length_factor, nom, fontsize=10)
    plt.arrow(0, 0, i* arrow_length_factor, j* arrow_length_factor, color = 'purple', alpha=0.7, width = 0.001)

plt.axis((-0.4, 0.4, -0.4, 0.4))
plt.gcf().gca().add_artist(plt.Circle((0, 0), radius = 0.4, color = 'cornflowerblue', fill = False))

plt.title('Carte de projection - PCA')
plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 2')
plt.grid(True)
plt.show()

L'ACP nous dit donc que l'on peut expliquer les jours de la semaine avec deux variables au lieu de 7, pour expliquer + de 90% de la variance. Cependant comme pour l'ACP précédente les affichages sont pas jolis. 
En R les variables sont plus interprétables et s'adaptent aux dimensions projetées. En Python, les résultats sont inaterprétables. 

Les projections dans les plans d'ACP (en R) nous permettent de faire l'hypothèses suivantes : 
- la composante 1 semble correspondre au chargement des stations comme dans le jeu de données complet. 
- la composante 2 semble faire la distinction semaine/week-end

Cependant, les variables ne sont pas très bien projeté sur la dimension 2 donc cela reste à nuancer. Les variables étaient mieux projetées sur la dimension 2 en utilisant le jeu de données complet.

#### Clustering

#### 4.1.1. Méthode de clustering avec k-means


L'algorithme K-Means est une technique de classification non-supervisée des données. Il cherche à minimiser les distances entre les observations et les centres des clusters auxquels elles appartiennent.

Tout d'abord, on va effectuer l'algorithme K-Means sur le jeu de données complet 'data_jours' et voir si on peut en tirer quelques interprétations.

On va déterminer le nombre de clusters optimal grace à la méthode du coude.

In [ ]:
inertia = []
for k in range(1, 15):
    kmeans_jours = KMeans(n_clusters=k, init='random', n_init='auto', max_iter=100, random_state=42)
    kmeans_jours.fit(data_jours)
    inertia.append(kmeans_jours.inertia_)
inertia = np.array(inertia)


plt.scatter(range(2, 15), inertia[1:])
plt.xlabel('nombre de clusters', fontsize = 15)
plt.ylabel('inertie', fontsize = 15)
plt.title("Graphe du coude", fontsize = 15)
plt.show()


# --- #


#avec yellowbrick

kmeans_jours = KMeans(init='random', n_init='auto', max_iter=100, random_state=42)
visualizer = KElbowVisualizer(kmeans_jours, k=(1,12))

visualizer.fit(data_jours)    
visualizer.show()    

D'après les graphes précedents, on peut prendre 3 ou 4. Pour la suite, on va probablement choisir 3 clusters. 

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15,8))

for k in range(2, 6):
    kmeans_jours = KMeans(n_clusters=k, init='random', n_init='auto', max_iter=100, random_state=42)
    q, mod = divmod(k, 2) 
    visualizer = SilhouetteVisualizer(kmeans_jours, colors='yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(data_jours)

Ce résultat est vérifié par les graphes du score silhouette ci-dessous car pour 3 clusters il y a peu de valeurs négatives et que les bandes dépassent le trait rouge. On choisira pour la suite de l'analyse 3 clusters. 

In [ ]:
K = 3
kmeans_jours = KMeans(n_clusters=K, init='random', n_init='auto', random_state=0)
clusters_jours = kmeans_jours.fit_predict(data_jours)
#On fixe le nombre de clusters à 3

Observons mainteant la répartition des valeurs dans chaque clusters. Pour voir si certains clusters sont plus importants que d'autres.

In [ ]:
cmap = plt.get_cmap('Set3',K)
plt.bar(*np.unique(clusters_jours, return_counts=True), color=cmap.colors)
plt.ylabel("Frequency")
plt.show()

La répartition des valeurs est très hétérogène, il n'y a que très peu de valeurs dans le 3ème cluster.

Avant de tirer des conclusions des analyses qui vont suivre, il faut vérifier que la variance intra-classe de chaque clusters.

In [ ]:
# distance de chaque point de données à son centre de cluster
distances = kmeans_jours.transform(data_jours)

#variance intraclasse pour chaque cluster
variances = []
for cluster in range(kmeans_jours.n_clusters):
    cluster_distances = distances[kmeans_jours.labels_ == cluster, cluster]
    variance = np.var(cluster_distances)
    variances.append(variance)

# Affichage
plt.figure(figsize=(8, 6))
plt.bar(range(kmeans_jours.n_clusters), variances, color='skyblue')
plt.xlabel('Clusters')
plt.ylabel('Variance intra-classe')
plt.title('Variance intra-classe par cluster')
plt.xticks(range(kmeans_jours.n_clusters), [f'Cluster {i+1}' for i in range(kmeans_jours.n_clusters)])
plt.show()


On voit que la variance des clusters est très petite, nous pouvons continuer nos analyses. <br>
On affiche maintenant les boxplots qui illustre la répartition des distances de chaque point au centre de son cluster. Nous allons vérifier que cette distribution est dans une fenêtre assez réduite.

In [ ]:
# Calculer les distances de chaque point de données à son centre de cluster
distances = kmeans_jours.transform(pd.DataFrame(data_jours))

# Créer une liste de distances pour chaque cluster
distances_per_cluster = [[] for _ in range(kmeans_jours.n_clusters)]
for cluster in range(kmeans_jours.n_clusters):
    cluster_distances = distances[kmeans_jours.labels_ == cluster, cluster]
    distances_per_cluster[cluster] = cluster_distances

# Plotter les boxplots
plt.figure(figsize=(10, 6))
plt.boxplot(distances_per_cluster, patch_artist=True, labels=[f'Cluster {i+1}' for i in range(kmeans_jours.n_clusters)])
plt.xlabel('Clusters')
plt.ylabel('Distance au centre de cluster')
plt.title('Distribution des distances au centre de cluster')
plt.show()





Les boxplots associés à chaque clusters ne sont pas très longs, cela est une autre façon d'illustrer les résultats précédents.

On va afficher maintenant le chargement moyen des stations par clusters au cours de la semaine. Cela va peut-être nous permettre de visualiser des tendances communes entre les éléments d'un même cluster.

In [ ]:
loading_sub3 = loading.iloc[:,:]
loading_sub3['cluster']=clusters_jours
mean_loading3=loading_sub3.groupby('cluster').mean()
mean_loading3.head()

fig, ax = plt.subplots(figsize=(10, 6))
col = ['#800080','#008080','#FFFF00']

# Affichage des 4 lignes sur le même graphique
for i in range(3):
    ax.plot(mean_loading3.columns, mean_loading3.iloc[i], label=f'Cluster {i+1}',color=col[i])

plt.legend()
plt.title('Moyenne des chargements par cluster')
plt.xlabel('Temps en heures')
plt.ylabel('Moyenne des loading')
plt.show()


Sur le dataframe des données regoupées par jours, les interprétations des clusters sont différentes. On peut voir que les clusters se différencient surtout sur leur moyenne de chargement. \
Le cluster 2 correspond aux stations peu chargées en moyenne. \
Le cluster 3 correspond aux stations moyennement chargées.\
Enfin, le cluster 1 correspond lui, aux stations chargées.\
On voit peu de différence entre la semaine/weekend sur le profil de chargement comparé à data_heures (et data_heures_pca) où la moyenne de chargement des clusters étaient complétement différentes en fonction de l'heure. 
Cela confirme notre analyse descriptive : le week-end a peu d'influence sur le chargement des stations. Cela a également pu se voir par le peu d'information que porte la composante 2 dans l'acp sur data_jours (8,2%)

On va maintenant tenter de faire un lien entre la variable 'Hill' est les clusters formés par K-Means. Pour ce faire, on dresse une table croisée qui va mettre en lien 'Hill' et les clusters. 

In [ ]:
tbl3 = pd.crosstab(clusters_jours, coord['bonus'])

#on affiche la mosaïque :
tbl3.index = ['Cluster_1', 'Cluster_2', 'Cluster_3'] 
tbl3.columns = ['Pas colline', 'Colline']  

sns.heatmap(tbl3, cmap='coolwarm', annot=True, fmt="d")
plt.title("Stations en altitude en fonction du clusters")
plt.show()

Les stations en altitude sont, dans ce cas aussi, regroupées globalement dans un seul cluster (ici le 2ème). En effet, 125 stations en altitudes sont contenues dans le 2ème cluster. Ce cluster correspond aux stations les plus souvent vides. Cela peut s'interpreter par le fait que beaucoup de personnes sur les collines empruntent un vélo pour descendre mais elles ont tendance à ne pas monter la colline en vélo. Les stations en altitude sont donc vides car les gens ne ramènent pas les vélos.

In [ ]:
# Etude sur data_jours_pca

Maintenant, on va effectuer l'algorithme K-Means à nouveau mais cette fois-ci, sur le jeu de données réduit par l'ACP. Cela va permettre de voir si les données réduites représentent bien le jeu de données complet. On cherche entre autre à voir si la classification réalisée sur le jeu de données complet et la même sur celui réduit, si c'est le cas cela nous permettra de simplifier les calculs des algorithmes suivants en travaillant sur le jeu de données réduit.

Déterminons le nombre de cluster optimal :

In [ ]:
inertia = []
for k in range(1, 15):
    kmeans_PCA_jours = KMeans(n_clusters=k, init='random', n_init='auto', max_iter=100, random_state=42)
    kmeans_PCA_jours.fit(data_jours_pca )
    inertia.append(kmeans_PCA_jours.inertia_)
inertia = np.array(inertia)


plt.scatter(range(2, 15), inertia[1:])
plt.xlabel('nombre de clusters', fontsize = 15)
plt.ylabel('inertie', fontsize = 15)
plt.title("Graphe du coude", fontsize = 15)
plt.show()


# --- #


# avec yellowbrick

kmeans_PCA_jours = KMeans(init='random', n_init='auto', max_iter=100, random_state=42)
visualizer = KElbowVisualizer(kmeans_PCA_jours, k=(2,12))

visualizer.fit(data_jours_pca )   
visualizer.show()    

D'après les graphes précedents on pourrait choisir un nombre de clusters de 4.  

Nous allons vérifier ce résultat par l'analyse des scores silhouette.

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15,8))

for k in range(2, 6):
    kmeans_PCA_jours = KMeans(n_clusters=k, init='random', n_init='auto', max_iter=100, random_state=42)
    q, mod = divmod(k, 2) 
    # Create SilhouetteVisualizer instance with KMeans instance
    visualizer = SilhouetteVisualizer(kmeans_PCA_jours, colors='yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(data_jours_pca )

Ces score silhouettes montrent que 3 ou 4 clusters est un bon choix, il y a pour les deux très peu de valeurs négatives et les bandes dépassent le trait rouge. Pour tenter de coller à l'analyse des données non réduites, on va choisir 3 clusters. 

In [ ]:
K = 3
kmeans_PCA_jours = KMeans(n_clusters=K, init='random', n_init='auto', random_state=0)
clusters_jours_pca = kmeans_PCA_jours.fit_predict(data_jours_pca )
#On fixe le nombre de cluster à 3

In [ ]:
#reclassement des clusters pour matcher les couleurs et faire correspondre les clusters 
cm2, clusters_jours_pca_sorted = matchClasses(clusters_jours, clusters_jours_pca)

Observons maintenant la répartition des valeurs dans chaque cluster. Pour voir si certains clusters sont plus importants que d'autres et si cela correspond à l'analyse des données complètes.

In [ ]:
cmap = plt.get_cmap('Set3',K)
plt.bar(*np.unique(clusters_jours_pca_sorted, return_counts=True), color=cmap.colors)
plt.ylabel("Frequency")
plt.show()

La répartition des valeurs est la même que précédemment, le cluster 3 ne possède pas beaucoup de valeurs comparé aux deux autres.

Avant de tirer des conclusions des analyses qui vont suivre, il faut vérifier que la variance intra-classe de chaque clusters.

In [ ]:
# distance de chaque point de données à son centre de cluster
distances = kmeans_PCA_jours.transform(data_jours_pca)

#variance intraclasse pour chaque cluster
variances = []
for cluster in range(kmeans_PCA_jours.n_clusters):
    cluster_distances = distances[kmeans_PCA_jours.labels_ == cluster, cluster]
    variance = np.var(cluster_distances)
    variances.append(variance)

# Affichage
plt.figure(figsize=(8, 6))
plt.bar(range(kmeans_PCA_jours.n_clusters), variances, color='skyblue')
plt.xlabel('Clusters')
plt.ylabel('Variance intra-classe')
plt.title('Variance intra-classe par cluster')
plt.xticks(range(kmeans_PCA_jours.n_clusters), [f'Cluster {i+1}' for i in range(kmeans_PCA_jours.n_clusters)])
plt.show()


La variance intra-classe est assez petite, nous choisissons de poursuivre les analyses.

In [ ]:
import matplotlib.pyplot as plt

# Définition des couleurs des clusters
colors = ['#800080','#008080','#FFFF00']

plt.figure(figsize=(8, 6))
# Tracé du nuage de points des clusters dans l'espace d'ACP
scatter = plt.scatter(loading_pca[:, 0], loading_pca[:, 1], c=clusters_jours_pca, cmap='viridis', s=50, alpha=0.5)

# Définition des étiquettes de légende pour chaque cluster
legend_labels = ['Cluster 1', 'Cluster 2', 'Cluster 3']

# Création des marqueurs de légende avec les couleurs correspondantes
legend_markers = [plt.Line2D([0], [0], linestyle='none', marker='o', color=color, markersize=10) for color in colors]

# Affichage de la légende avec les marqueurs et les étiquettes définis
plt.legend(legend_markers, legend_labels, loc='lower right')

plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Clusters dans le plan de l\'ACP')
plt.show()

Les valeurs sont bien réparties entre les clusters et ne se mélangent pas.

Nous allons maintenant voir si, dans ce cas aussi, la variable 'Hill' a un comportement particulier avec les clusters formés par K-Means. 

In [ ]:
tbl3_1 = pd.crosstab(clusters_jours_pca_sorted, coord['bonus'])

#on affiche la mosaïque :
tbl3_1.index = ['Cluster_1', 'Cluster_2', 'Cluster_3'] 
tbl3_1.columns = ['Pas colline', 'Colline']  

sns.heatmap(tbl3_1, cmap='coolwarm', annot=True, fmt="d")
plt.title("Stations en altitude en fonction du clusters")
plt.show()

Dans cette classification aussi, un cluster contient la majorité des stations en altitude. On peut faire les mêmes déductions que pour le Kmeans sur data_jours.

On va afficher maintenant le chargement moyen des stations par cluster au cours de la semaine. Cela va peut-être nous permettre de visualiser des tendances communes entre les éléments d'un même cluster.

In [ ]:
loading_sub4 = loading.iloc[:,:]
loading_sub4['cluster']=clusters_jours_pca_sorted
mean_loading4=loading_sub4.groupby('cluster').mean()
mean_loading4.head()

fig, ax = plt.subplots(figsize=(10, 6))
col = ['#800080','#008080','#FFFF00']

# Affichage des 4 lignes sur le même graphique
for i in range(3):
    ax.plot(mean_loading4.columns, mean_loading4.iloc[i], label=f'Cluster {i+1}', color= col[i])

plt.legend()
plt.title('Moyenne des chargements par cluster')
plt.xlabel('Temps en heures')
plt.ylabel('Moyenne des loading')
plt.show()


Comme précédemment, on remarque un cluster qui représente les stations non chargées, un autre les stations moyennement chargées et enfin celles qui le sont le plus.

Comparaison des deux classifications :

On compare les deux classifications grâce à un scatterplot pour voir le nombre de valeurs qui correspondent dans le cas normal et le cas réduit, afin de voir si il est vraiment utile de faire son analyse sur l'ACP ou non  :

In [ ]:
Position = np.array(coord)[:,0:2]

#graphe à pour data_jours entier
f1 = px.scatter_mapbox(clusters_jours, Position[:,1], Position[:,0], color=clusters_jours, color_continuous_scale=[(0, '#800080'), (0.5, '#008080'), (1, '#FFFF00')], 
                        size_max=15, zoom=10,mapbox_style="carto-positron", opacity = .9,
                        title = 'clusters données complètes')

f1.show()

#graphe à pour data_jours_pca
f2 = px.scatter_mapbox(clusters_jours_pca_sorted, Position[:,1], Position[:,0], color=clusters_jours_pca_sorted, color_continuous_scale =[(0, '#800080'), (0.5, '#008080'), (1, '#FFFF00')], 
                        size_max=15, zoom=10,mapbox_style="carto-positron", opacity = .9,
                        title = 'clusters données pca')
f2.show()



points_diff = clusters_jours != clusters_jours_pca_sorted

# Calcul du nombre de points différents
num_diff_points = np.sum(points_diff)

# Calcul du pourcentage de réussite
pourcentage_reussite = (1- num_diff_points / len(clusters_jours)) * 100

# Affichage du résultat
print(f"Nombre de points différents : {num_diff_points} sur {len(clusters_jours)}")
print(f"Pourcentage de réussite : {pourcentage_reussite:.2f}%")




Le pourcentage de réussite d'association des points est moins importante que pour l'étude précédente cependant il reste toujours assez important pour pouvoir considérer que les données réduites avec l'ACP réprésentent bien les données complètes.

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(clusters_jours, clusters_jours_pca)).plot()
plt.xlabel('On the reduced data (PCA)')
plt.ylabel('On the complete data')
plt.show()

On visualise bien grâce à ce graphe que la majorité des valeurs sont bien réparties dans les 3 clusters différents mais sont mélangées, il faut les reclasser.

In [ ]:
# reclassement des clusters

In [ ]:
ConfusionMatrixDisplay(cm2).plot()
plt.xlabel('On the reduced data (PCA)')
plt.ylabel('On the complete data')
plt.show()

Cette figure valide ce qui a été dit précedemment, soit que les données dans le plan réduit et dans le plan PCA correspondent bien. Pour la suite des analyses, on peut travailler uniquement sur les données réduit pour réduire le temps de calcul des algorithmes.

In [ ]:
#on convertit la table de confusion en dataframe 
df_cm2 = pd.DataFrame(cm2, columns=['groupe0', 'groupe1','groupe2'], index=['groupe_pca0', 'groupe_pca1','groupe_pca2',])
df_cm2.head()

En affichant la table de contingence, on voit que le nombre de stations mal classifié est très faible, cela nous permet de conclure (directement et non en passant par une CA) que nos méthodes de classification sont comparables. Ceci signifie que le  jeu de données complet et le jeu réduit ont des clusters qui correspondent bien, on va pouvoir considérer prendre les données réduites pour la suite de nos analyses pour faciliter les calculs.

#### 4.1.2. CAH : Agglomerative Clustering


Etudions ici la méthode de la classification ascendante hiérarchique (CAH). Cette méthode consiste à afficher des dendrogrammes des différente données en fonction de différents likages.

Comme expliqué précédemment, on préférera faire l'analyse en prenant un échantillon plus petit pour minimiser les temps de calculs. Nous avons vu que l'analyse des données PCA donnait de bons résultats, donc pour faciliter les calculs, lors des analyses suivantes nous prendrons les données jours réduites.

In [ ]:
print("Dimension du jeu de données entier 'loading' :" , data_jours.shape)
print("Dimension du jeu de données réduit 'loading_pca': " +str(data_jours_pca.shape))

Les données issues de la PCA sont donc bien réduites.

On détermine le nombre de clusters adéquats grâce au graphe du coude :

In [ ]:
#Réalisation du CAH avec le linkage 'Ward' 
ac = AgglomerativeClustering(linkage="ward", compute_distances=True)
clusters_cah2 = ac.fit_predict(data_jours_pca)

#Méthode d'affichage de sélection des clusters
ac = AgglomerativeClustering(linkage='ward', compute_distances=True)
visualizer = KElbowVisualizer(ac, k=(2,12))

visualizer.fit(data_jours_pca) 
visualizer.show()   
plt.show()

D'après les résultats graphiques obtenus en R et en Python, nous décidons de prendre 3 clusters différents.

On affiche alors le dendrogramme associé en utilisant le linkage "Ward".

In [ ]:
#Nombre de clusters choisi par les indicateurs 
K = 3

#Réalisation du CAH avec le linkage 'Ward' & affichage du résultat en coupant à K = 3 clusters
ac = AgglomerativeClustering(n_clusters=K, compute_distances=True, linkage='ward')
clusterscah2 = ac.fit(data_jours_pca)

children = ac.children_
distances = ac.distances_
n_observations = np.arange(2, children.shape[0]+2)
linkage_matrix = np.c_[children, distances, n_observations]

sch.dendrogram(linkage_matrix, labels=ac.labels_)

# On coupe le dendrogramme à K =3 clusters 
max_d = .5*(ac.distances_[-K]+ac.distances_[-K+1])
plt.axhline(y=max_d, c='k')

plt.title("Dendrogramme avec le linkage Ward")
plt.show()

On affiche tous les types de linkages que l'on a étudié pour montrer que "Ward" et "complete" sont les deux types dont les graphes sont faciles à explorer:

In [ ]:
#Affichage des dendrogrammes avec les différents linkages afin de pouvoir voir les différences sur un même jeu de données

plt.subplot(2,2,1)
linkage_matrix_single = sch.linkage(data_jours_pca, method='single')
sch.dendrogram(linkage_matrix_single)
plt.title("Dendrogramme avec le linkage simple")

plt.subplot(2,2,2)
linkage_matrix_complete = sch.linkage(data_jours_pca, method='complete')
sch.dendrogram(linkage_matrix_complete)
plt.title("Dendrogramme avec le linkage complete")

plt.subplot(2,2,3)
linkage_matrix_average = sch.linkage(data_jours_pca, method='average')
sch.dendrogram(linkage_matrix_average)
plt.title("Dendrogramme avec le linkage moyenne")

plt.subplot(2,2,4)
linkage_matrix_ward = sch.linkage(data_jours_pca, method='ward')
sch.dendrogram(linkage_matrix_ward)
plt.title("Dendrogramme avec le linkage Ward")

plt.tight_layout()
plt.show()

On voit que le graphe issu du linkage 'simple' est illisible, les autres sont mieux mais c'est sur le graphe du linkage 'Ward' qu'on voit le mieux le 'jump' important formant les 3 clusters de tailles assez similaires. Donc on garde le linkage 'Ward'. 

Les interprétations sur R nous donne les mêmes informations que les KMeans.

#### 4.1.3. Gaussian Mixture Models

On peut utiliser les données issues de l'ACP car elle résume bien notre jeu de données 'jours'.

Afin de mettre en place la méthode du GMM, on applique un critère de sélection du nombre de clusters. On choisit ici de travailler avec le crière BIC.

In [ ]:
# Critère BIC :

In [ ]:
k_max = 15

bic = []
for k in range(2, k_max):
    gmm = GaussianMixture(n_components=k, init_params='kmeans', n_init=3)
    gmm.fit(data_jours_pca)
    bic.append(gmm.bic(data_jours_pca))
bic = np.array(bic)

plt.scatter(range(2, k_max), bic)
plt.show()

En appliquant le critère BIC on devrait garder 5 variables pour expliquer nos données en Python et on trouve 4 en R. Ce qui se rapproche globalement de ce que l'on a trouvé par la méthode du coude précédemment. On décide de garder 3 variables pour être en accord avec le nombre de clusters dans Kmeans et faire une interprétation globale.

On affiche d'abord la dispersion des points par cluster initiée par le modèle de mélange gaussien (GMM)

In [ ]:
K = 3
cmap = plt.get_cmap('Set3', K)

gmm = GaussianMixture(n_components=K, n_init=3)
clusters_gmm_jours = gmm.fit_predict(data_jours_pca)

# --- #


# Définition des couleurs des clusters
colors = ['#800080','#008080','#FFFF00']

plt.figure(figsize=(8, 6))
# Tracé du nuage de points des clusters dans l'espace d'ACP
scatter = plt.scatter(loading_pca[:, 0], loading_pca[:, 1], c=clusters_gmm_jours, cmap='viridis', s=50, alpha=0.5)

# Définition des étiquettes de légende pour chaque cluster
legend_labels = ['Cluster 1', 'Cluster 2', 'Cluster 3']

# Création des marqueurs de légende avec les couleurs correspondantes
legend_markers = [plt.Line2D([0], [0], linestyle='none', marker='o', color=color, markersize=10) for color in colors]

# Affichage de la légende avec les marqueurs et les étiquettes définis
plt.legend(legend_markers, legend_labels, loc='lower right')

plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Clusters dans le plan de l\'ACP')
plt.show()

On obtient des clusters très surprenants par rapport à ceux obtenus sur les autres méthodes de classifications ou les autres jeux de données.

Avant de continuer nos analyses
, on va vérifier que la variance dans chaque cluster ne soit pas trop grande, ainsi les élèments d'un même cluster se comporteraient en moyenne de la même manière. <br> On affiche pour cela les boxplots des données de chaque cluster.

In [ ]:
aux = pd.DataFrame({
    'label': ['Cluster ' + str(label) for label in clusters_gmm_jours],
    'proba': np.max(gmm.predict_proba(data_jours_pca), axis=1)
})
sns.boxplot(x='label', y='proba', data=aux, palette='Set3', showfliers=False)
plt.title('Cluster Probabilities')
plt.show()


Les variances des données dans chaque cluster, représentée par la longueur des boxplots, est relativement faible, on peut continuer nos analyses.

On affiche maintenant le chargement moyen des stations par cluster (issus de GMM) au cours de la semaine. Cela va peut-être nous permettre de visualiser des tendances communes entre les éléments d'un même cluster.

In [ ]:
loading_sub5_2 = loading.iloc[:,:]
loading_sub5_2['cluster']=clusters_gmm_jours
mean_loading5=loading_sub5_2.groupby('cluster').mean()
mean_loading5.head()


fig, ax = plt.subplots(figsize=(10, 6))
col= ['purple', 'blue'  ,'green']

# Affichage des 4 lignes sur le même graphique

for i in range(3):
    ax.plot(mean_loading5.columns, mean_loading5.iloc[i], label=f'Cluster {i+1}',color=col[i])

    
plt.legend()
plt.title('Moyenne des chargements par cluster')
plt.xlabel('Temps en heures')
plt.ylabel('Moyenne des loading')
plt.show()

On affiche désormais le plot mosaïque, qui permet de voir combien de stations de chaque cluster sont en altitude ou non:

In [ ]:
tbl2_3 = pd.crosstab(clusters_gmm_jours, coord['bonus'])

#on affiche la mosaïque :
tbl2_3.index = ['Cluster_1', 'Cluster_2', 'Cluster_3'] 
tbl2_3.columns = ['Pas colline', 'Colline']  

sns.heatmap(tbl2_3, cmap='coolwarm', annot=True, fmt="d")
plt.title("Stations en altitude en fonction du clusters")
plt.show()


Notre plot mosaïque nous permet de dire qu'un clutser contient les stations localisées sur des collines, ce qui rejoint le graphe des moyennes de chargment où le chargement est assez bas. Cela rejoint ce que l'on a dit sur l'analyse de Kmeans. Concernant le chargement moyen des différents clusters et sur le fait que les stations en altitude sont toutes dans le cluster 3 qui correspond aux stations faiblement chargées en moyenne.
Cependant, dans cette méthode le cluster_3 comporte plus de variations : il a tendance à se charger la nuit et se vider d'un coup le matin les jours de semaine.

Réalisons maintenant une étude comparative des différentes méthodes de clusters : 

In [ ]:
cm2_GMM, clusters_pca_sorted_heures_GMM = matchClasses(clusters_gmm_jours, clusters_jours_pca)


ConfusionMatrixDisplay(cm2_GMM).plot()

plt.xlabel('Kmeans sur les données jours réduites (PCA)')
plt.ylabel('Gmm sur les données jours réduites ')
plt.show()

df_kmeans_gmm2 = pd.DataFrame(cm2_GMM, columns=['groupe0', 'groupe1','groupe2'], index=['groupe_pca0', 'groupe_pca1','groupe_pca2'])
df_kmeans_gmm2.head()

En affichant la table de contingence, on voit que le nombre de stations mal classifié est très faible, cela nous permet de conclure (directement et non en passant par une CA) que nos méthodes de classification sont comparables.

En R, on réalise égalemetn une Analyse en Correspondance entre les méthodes CAH et Kmeans pour ce jeu de données data_jours, afin de comparer les méthodes de classifications. On peut voir que les clusters entre K-Means et CAH sur data_jours sont quasiment les mêmes alors que entre GMM et K-Means les clusters ne sont pas exactement pareil, d'où la différence par exemple entre la distribution des stations en altitude dans les clusters.

## 5. Etude sur le jeu de données par heures

Nous réalisons encore une fois un nouveau dataframe en regroupant les taux moyens par heures de la journée.

### 5.1. ACP

Afin de simplifier la dimension de notre dataframe et déduire de potentiels liens entre les taux de chargement et le , on souhaite réaliser une Analyse en composantes principales (ACP). 

Comme précédemment, il n'est pas nécessaire de standardiser les données car celles-ci sont comprises entre 0 et 1 en tant que taux de chargement. Il n'est donc pas nécessaire de standardiser les données.

In [ ]:
# Création du nouveau dataset 

#Etape 1: 
# Calcul des moyennes pour chaque heure de la journée
minuit_am = loading.iloc[:, [0, 24, 48, 72, 96, 120, 144]].mean(axis=1)
une_am = loading.iloc[:, [1, 25, 49, 73, 97, 121, 145]].mean(axis=1)
deux_am = loading.iloc[:, [2, 26, 50, 74, 98, 122, 146]].mean(axis=1)
trois_am = loading.iloc[:, [3, 27, 51, 75, 99, 123, 147]].mean(axis=1)
quatre_am = loading.iloc[:, [4, 28, 52, 76, 100, 124, 148]].mean(axis=1)
cinq_am = loading.iloc[:, [5, 29, 53, 77, 101, 125, 149]].mean(axis=1)
six_am = loading.iloc[:, [6, 30, 54, 78, 102, 126, 150]].mean(axis=1)
sept_am = loading.iloc[:, [7, 31, 55, 79, 103, 127, 151]].mean(axis=1)
huit_am = loading.iloc[:, [8, 32, 56, 80, 104, 128, 152]].mean(axis=1)
neuf_am = loading.iloc[:, [9, 33, 57, 81, 105, 129, 153]].mean(axis=1)
dix_am = loading.iloc[:, [10, 34, 58, 82, 106, 130, 154]].mean(axis=1)
onze_am = loading.iloc[:, [11, 35, 59, 83, 107, 131, 155]].mean(axis=1)
minuit_pm = loading.iloc[:, [12, 36, 60, 84, 108, 132, 156]].mean(axis=1)
une_pm = loading.iloc[:, [13, 37, 61, 85, 109, 133, 157]].mean(axis=1)
deux_pm = loading.iloc[:, [14, 38, 62, 86, 110, 134, 158]].mean(axis=1)
trois_pm = loading.iloc[:, [15, 39, 63, 87, 111, 135, 159]].mean(axis=1)
quatre_pm = loading.iloc[:, [16, 40, 64, 88, 112, 136, 160]].mean(axis=1)
cinq_pm = loading.iloc[:, [17, 41, 65, 89, 113, 137, 161]].mean(axis=1)
six_pm = loading.iloc[:, [18, 42, 66, 90, 114, 138, 162]].mean(axis=1)
sept_pm = loading.iloc[:, [19, 43, 67, 91, 115, 139, 163]].mean(axis=1)
huit_pm = loading.iloc[:, [20, 44, 68, 92, 116, 140, 164]].mean(axis=1)
neuf_pm = loading.iloc[:, [21, 45, 69, 93, 117, 141, 165]].mean(axis=1)
dix_pm = loading.iloc[:, [22, 46, 70, 94, 118, 142, 166]].mean(axis=1)
onze_pm = loading.iloc[:, [23, 47, 71, 95, 119, 143, 167]].mean(axis=1)

#Etape 2 : on forme le jeu de données 
# Création du nouveau DataFrame
data_heures = pd.DataFrame({
    'Minuit': minuit_am,
    '1h': une_am,
    '2h': deux_am,
    '3h': trois_am,
    '4h': quatre_am,
    '5h': cinq_am,
    '6h': six_am,
    '7h': sept_am,
    '8h': huit_am,
    '9h': neuf_am,
    '10h': dix_am,
    '11h': onze_am,
    '12h': minuit_pm,
    '13h': une_pm,
    '14h': deux_pm,
    '15h': trois_pm,
    '16h': quatre_pm,
    '17h': cinq_pm,
    '18h': six_pm,
    '19h': sept_pm,
    '20h': huit_pm,
    '21h': neuf_pm,
    '22h': dix_pm,
    '23h': onze_pm
})

data_heures['Hill'] = coord['bonus']

print(data_heures)

In [ ]:
pca_3 = PCA()
data_heures_pca = pca_3.fit_transform(data_heures)

In [ ]:
#On affiche la variance expliquée par les composantes de l'ACP
plt.plot(np.cumsum(pca_3.explained_variance_ratio_))

plt.title('Variance cumulée expliquée en fonction de la dimension de l espace de l`ACP')
plt.xlabel('Nombre de composantes dans l ACP')
plt.ylabel('Variance cumulée expliquée')

#On regarde le nombre de composantes nécessaires pour expliquer 90 % de la variance 
pca_3 = PCA(0.90).fit(data_heures) 
pca_3.n_components_ 
print(f"on garde {pca_3.n_components_} composantes pour le PCA")

La fonction nous dit qu'il faut garder 3 composantes pour expliquer 90% de la variance.

In [ ]:
#On réalise l'ACP pour 3 composantes
pca_3 = PCA(n_components = 3)
data_heures_pca = pca_3.fit_transform(data_heures) 

print(100*pca_3.explained_variance_ratio_)

#Affichage et vérification de la dimension du jeu de données avant et après ACP
print('--- PCA ---')
print('Dimension initiale :' , data_heures.shape)
print('Dimension après projection:', data_heures_pca.shape)

print('')

print('--- Variance expliquée  ---')
print('Composante 1:', round(pca.explained_variance_[0],2), 'i.e.', round(100*pca.explained_variance_ratio_[0],2), '% de la variance totale')
print('Composante 2:', round(pca.explained_variance_[1],2), 'i.e.', round(100*pca.explained_variance_ratio_[1],2), '% de la variance totale')
print('Composante 3:', round(pca.explained_variance_[2],2), 'i.e.', round(100*pca.explained_variance_ratio_[2],2), '% de la variance totale')

In [ ]:
#Affichage des bar plot de la proportion d'explication de la variance par chaque composante de l'ACP
plt.figure(figsize=(10, 5))
sns.barplot(x=[f"PC{i+1}" for i in range(len(pca_3.explained_variance_ratio_))], y=pca_3.explained_variance_ratio_ * 100)
plt.xlabel("Composantes Principales")
plt.ylabel("Pourcentage de Variance Expliquée")
plt.title("Proportion de Variance Expliquée par Chaque Composante Principale")
plt.ylim(0,80) 
plt.show()

In [ ]:
#Affichage des boxplot des composantes de l'ACP
box = plt.boxplot(data_heures_pca[:,:10], patch_artist=True)
plt.setp(box["boxes"], facecolor="coral", alpha=.5)
plt.title("Box plots des 3 premières composantes")
plt.tight_layout()
plt.show()

On projette les données triées par heures sur les deux premiers axes de l'ACP.

In [ ]:
coord1 = pca.components_[0] * np.sqrt(pca_3.explained_variance_[0])
coord2 = pca.components_[1] * np.sqrt(pca_3.explained_variance_[1])

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(1, 1, 1)
for i, j, nom in zip(coord1, coord2, data_heures.columns):
    plt.text(i, j, nom, fontsize=10)
    plt.arrow(0, 0, i, j, color = 'purple', alpha=0.7, width = 0.0001)

plt.axis((-0.2, 0.2, -0.2, 0.2))
plt.gcf().gca().add_artist(plt.Circle((0, 0), radius = 0.2, color = 'cornflowerblue', fill = False))

plt.title('Carte de projection - PCA')
plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 2')
plt.grid(True)
plt.show()

On projette les données triées par heures sur l'axes 1 et 3 de l'ACP.

In [ ]:
coord1 = pca.components_[0] * np.sqrt(pca_3.explained_variance_[0])
coord2 = pca.components_[2] * np.sqrt(pca_3.explained_variance_[2])

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(1, 1, 1)
for i, j, nom in zip(coord1, coord2, data_heures.columns):
    plt.text(i, j, nom, fontsize=10)
    plt.arrow(0, 0, i, j, color = 'purple', alpha=0.7, width = 0.0001)

plt.axis((-0.2, 0.2, -0.2, 0.2))
plt.gcf().gca().add_artist(plt.Circle((0, 0), radius = 0.2, color = 'cornflowerblue', fill = False))

plt.title('Carte de projection - PCA')
plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 3')
plt.grid(True)
plt.show()

On projette les données triées par heures sur l'axes 2 et 3 de l'ACP.

In [ ]:
coord1 = pca.components_[1] * np.sqrt(pca_3.explained_variance_[1])
coord2 = pca.components_[2] * np.sqrt(pca_3.explained_variance_[2])

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(1, 1, 1)
for i, j, nom in zip(coord1, coord2, data_heures.columns):
    plt.text(i, j, nom, fontsize=10)
    plt.arrow(0, 0, i, j, color = 'purple', alpha=0.7, width = 0.0001)

plt.axis((-0.15, 0.15, -0.15, 0.15))
plt.gcf().gca().add_artist(plt.Circle((0, 0), radius = 0.15, color = 'cornflowerblue', fill = False))

plt.title('Carte de projection - PCA')
plt.xlabel('Composante principale 1')
plt.ylabel('Composante principale 3')
plt.grid(True)
plt.show()

ANALYSE : 
Voir le graphe des variables en R.

Dans ce nouveau dataset et grâce aux projections dans les plans d'ACP, on peut faire les hypothèses suivantes : 
- la composante 1 semble toujours représenter le chargement des stations.
- la composante 2 semble elle correspondre aux heures de nuit et de jour. 
- la composante 3 semble évoquer les heures ou le chargement varie le plus, mais les variables ne sont pas bien projetées donc cela reste à nuancer. 

Les variables sont mieux projetées sur la dimension 2 en utilisant ce jeu de données que celui par jour. 

#### Clustering

#### 5.1.1. Méthode de clustering avec k-means


L'algorithme K-Means est une technique de classification non-supervisée des données. Il cherche à minimiser les distances entre les observations et les centres des clusters auxquels elles appartiennent.

Tout d'abord, on va effectuer l'algorithme K-Means sur le jeu de données complet 'data_heures' et voir si on peut en tirer quelques interprétations.

On va déterminer le nombre de clusters optimal grace à la méthode du coude.

In [ ]:
inertia = []
for k in range(1, 15):
    kmeans_heures = KMeans(n_clusters=k, init='random', n_init='auto', max_iter=100, random_state=42)
    kmeans_heures.fit(data_heures )
    inertia.append(kmeans_heures.inertia_)
inertia = np.array(inertia)


plt.scatter(range(2, 15), inertia[1:])
plt.xlabel('nombre de clusters', fontsize = 15)
plt.ylabel('inertie', fontsize = 15)
plt.title("Graphe du coude", fontsize = 15)
plt.show()


# --- #


# Using yellowbrick

kmeans_heures = KMeans(init='random', n_init='auto', max_iter=100, random_state=42)
visualizer = KElbowVisualizer(kmeans_heures, k=(1,12))

visualizer.fit(data_heures )    # Fit the data to the visualizer
visualizer.show()    # Finalize and render the figure

Comme pour l'analyse des autres jeux de données, le nombre de clusters idéal est 4. Avec les graphes de scores silhouettes suivantt on peut valider ce résultat. 

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15,8))

for k in range(2, 6):
    kmeans_heures = KMeans(n_clusters=k, init='random', n_init='auto', max_iter=100, random_state=42)
    q, mod = divmod(k, 2) 
    # Create SilhouetteVisualizer instance with KMeans instance
    visualizer = SilhouetteVisualizer(kmeans_heures, colors='yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(data_heures )

En effet, le choix d'un nombre de cluster de 4 semble optimal car le score silhouette associé ne possède quasi pas de valeurs négatives et toutes les bandes dépassent le trait rouge.

In [ ]:
K = 4
kmeans_heures = KMeans(n_clusters=K, init='random', n_init='auto', random_state=0)
clusters_heures = kmeans_heures.fit_predict(data_heures)
#On fixe le nombre de clusters à 4

Observons mainteant la répartition des valeurs dans chaque cluster. Pour voir si certains clusters sont plus importants que d'autres.

In [ ]:
cmap = plt.get_cmap('Set3',K)
plt.bar(*np.unique(clusters_heures, return_counts=True), color=cmap.colors)
plt.ylabel("Frequency")
plt.show()

Les valeurs sont réparties équitablement dans les clusters 1, 2 et 4 à la différence du 3 qui possède davantage de valeurs. 

Avant de tirer des conclusions des analyses qui vont suivre, il faut vérifier que la variance intra-classe de chaque clusters.

In [ ]:
# distance de chaque point de données à son centre de cluster
distances = kmeans_heures.transform(data_heures)

#variance intraclasse pour chaque cluster
variances = []
for cluster in range(kmeans_heures.n_clusters):
    cluster_distances = distances[kmeans_heures.labels_ == cluster, cluster]
    variance = np.var(cluster_distances)
    variances.append(variance)

# Affichage
plt.figure(figsize=(8, 6))
plt.bar(range(kmeans_heures.n_clusters), variances, color='skyblue')
plt.xlabel('Clusters')
plt.ylabel('Variance intra-classe')
plt.title('Variance intra-classe par cluster')
plt.xticks(range(kmeans_heures.n_clusters), [f'Cluster {i+1}' for i in range(kmeans_heures.n_clusters)])
plt.show()


On voit que la variance des clusters est raisonnable, nous pouvons continuer nos analyses. <br>
On affiche maintenant les boxplots qui illustre la répartition des distances de chaque point au centre de son cluster. Nous allons vérifier que cette distribution est dans une fenêtre assez réduite.

In [ ]:
# Calculer les distances de chaque point de données à son centre de cluster
distances = kmeans_heures.transform(pd.DataFrame(data_heures))

# Créer une liste de distances pour chaque cluster
distances_per_cluster = [[] for _ in range(kmeans_heures.n_clusters)]
for cluster in range(kmeans_heures.n_clusters):
    cluster_distances = distances[kmeans_heures.labels_ == cluster, cluster]
    distances_per_cluster[cluster] = cluster_distances

# Plotter les boxplots
plt.figure(figsize=(10, 6))
plt.boxplot(distances_per_cluster, patch_artist=True, labels=[f'Cluster {i+1}' for i in range(kmeans_heures.n_clusters)])
plt.xlabel('Clusters')
plt.ylabel('Distance au centre de cluster')
plt.title('Distribution des distances au centre de cluster')
plt.show()





On voit que les boxplots ne sont pas très larges, on peut donc continuer nos analyses.

On va afficher maintenant le chargement moyen des stations par cluster au cours de la semaine. Cela va peut-être nous permettre de visualiser des tendances communes entre les éléments d'un même cluster.

In [ ]:
loading_sub5 = loading.iloc[:,:]
loading_sub5['cluster']=clusters_heures
mean_loading5=loading_sub5.groupby('cluster').mean()
mean_loading5.head()


fig, ax = plt.subplots(figsize=(10, 6))
col= ['#FF69B4', '#008080'  ,'#800080','#FFFF00']

# Affichage des 4 lignes sur le même graphique

for i in range(4):
    ax.plot(mean_loading5.columns, mean_loading5.iloc[i], label=f'Cluster {i+1}',color=col[i])

    
plt.legend()
plt.title('Moyenne des chargements par cluster')
plt.xlabel('Temps en heures')
plt.ylabel('Moyenne des loading')
plt.show()

Les tendances de courbes du chargement moyen de data_heures sont similaires à celles obtenues lors de l'étude de loading.
On retrouve un cluster qui à tendance à se vider le matin et à se remplir à nouveau le soir. Le second cluster correspond aux stations qui se chargent aussi le matin et se remplissent le soir mais qui auront tendances à rester assez remplies. 
Ensuite, on retrouve un cluster qui se charge la journée et se décharge la nuit. Le 4ème à un comportement similaire mais est plus souvent vide.

Nous allons vérifier que pour ce jeu de donnée également, il y a un lien entre la variable 'hill' et les clusters.

In [ ]:
tbl4_1 = pd.crosstab(clusters_heures, coord['bonus'])

#on affiche la mosaïque :
tbl4_1.index = ['Cluster_1', 'Cluster_2', 'Cluster_3','Cluster_4'] 
tbl4_1.columns = ['Pas colline', 'Colline']  

sns.heatmap(tbl4_1, cmap='coolwarm', annot=True, fmt="d")
plt.title("Stations en altitude en fonction du clusters")
plt.show

A nouveau, on voit que les stations en altitudes sont toutes regroupées dans 1 seul cluster, celui qui corespond aux taux de chargement les plus bas comme précédemment.

In [ ]:
# Etude sur data_heures_pca

Maintenant, on va effectuer l'algorithme K-Means à nouveau mais cette fois-ci, sur le jeu de données réduit par l'ACP. Cela va permettre de voir si les données réduites représentent bien le jeu de données complet. On cherche entre autre à voir si la classification réalisée sur le jeu de données complet et la même sur celui réduit, si c'est le cas cela nous permettra de simplifier les calculs des algorithmes suivants en travaillant sur le jeu de données réduit.

Déterminons le nombre de clusters optimal.

In [ ]:
inertia = []
for k in range(1, 15):
    kmeans_PCA_heures = KMeans(n_clusters=k, init='k-means++', n_init='auto', max_iter=100, random_state=42)
    kmeans_PCA_heures.fit(data_heures_pca )
    inertia.append(kmeans_PCA_heures.inertia_)
inertia = np.array(inertia)


plt.scatter(range(2, 15), inertia[1:])
plt.xlabel('nombre de clusters', fontsize = 15)
plt.ylabel('inertie', fontsize = 15)
plt.title("Graphe du coude", fontsize = 15)
plt.show()


# --- #


# avec yellowbrick

kmeans_PCA_heures = KMeans(init='random', n_init='auto', max_iter=100, random_state=42)
visualizer = KElbowVisualizer(kmeans_PCA_heures, k=(1,12))

visualizer.fit(data_heures_pca)   
visualizer.show()   

Le coude est ici à 4. Vérifions maintenant ce résultat grace aux graphiques des scores silhouette.

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15,8))

for k in range(2, 6):
    kmeans_PCA_heures = KMeans(n_clusters=k, init='random', n_init='auto', max_iter=100, random_state=42)
    q, mod = divmod(k, 2) 
    # Create SilhouetteVisualizer instance with KMeans instance
    visualizer = SilhouetteVisualizer(kmeans_PCA_heures, colors='yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(data_heures_pca )

Le résultat précédent est validé. Toutes les conditions nécessaires sont présentes.

In [ ]:
K = 4
kmeans_PCA_heures = KMeans(n_clusters=K, init='random', n_init='auto', random_state=0)
clusters_heures_pca = kmeans_PCA_heures.fit_predict(data_heures_pca )
#On fixe le nombre de clusters à 4

Quelles est la répartition des variables dans les clusters?

In [ ]:
cmap = plt.get_cmap('Set3',K)
plt.bar(*np.unique(clusters_heures_pca, return_counts=True), color=cmap.colors)
plt.ylabel("Frequency")
plt.show()

Les données sont distribuées de la même facon que pour les données data_heures complètes.

In [ ]:
plt.figure(figsize=(8, 6))
#affichage du nuage de points des clusters identifiés par des couleurs différentes
plt.scatter(data_heures_pca[:, 0], data_heures_pca[:, 1], c=clusters_heures_pca, cmap='viridis', s=50, alpha=0.5)
legend_labels = ['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4']  # Liste des étiquettes de légende pour chaque cluster
legend_markers = [plt.Line2D([0], [0], linestyle='none', marker='o', color='#800080', markersize=10),
                  plt.Line2D([0], [0], linestyle='none', marker='o', color='#008080', markersize=10),
                  plt.Line2D([0], [0], linestyle='none', marker='o', color='#FFFF00', markersize=10),
                  plt.Line2D([0], [0], linestyle='none', marker='o', color='#FF69B4', markersize=10)]

plt.legend(legend_markers, legend_labels, loc='lower right')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Clusters dans le plan de l\'ACP')
plt.show()



Ce graphe représente la répartition des données dans le plan de l'ACP (dim 1 et dim 2). On voit que les points sont bien réparties dans les clusters et qu'ils ne se mélangent pas.\
Les points du cluster de droite correspondent aux stations chargées en moyenne (car corrélées positivement avec la dimension 1) avec une variation faible (car peu corrélée selon la dimension 2).
Les points du cluster de gauche correspondent aux stations peu chargées en moyenne (car corrélées négativement avec la dimension 1) avec une variation faible (car peu corrélée selon la dimension 2).
Les points du cluster du haut correspondent aux stations fortement chargées la nuit (corrélées positivement avec la dimension 2) et peu chargées le jour.
Les points du cluster du bas correspondent aux stations fortement chargées le jour (corrélées négativement avec la dimension 2) et peu chargées la nuit.

Avant de tirer des conclusions des analyses qui vont suivre, il faut vérifier que la variance intra-classe de chaque clusters.

In [ ]:
# distance de chaque point de données à son centre de cluster
distances = kmeans_PCA_heures.transform(data_heures_pca)

#variance intraclasse pour chaque cluster
variances = []
for cluster in range(kmeans_PCA_heures.n_clusters):
    cluster_distances = distances[kmeans_PCA_heures.labels_ == cluster, cluster]
    variance = np.var(cluster_distances)
    variances.append(variance)

# Affichage
plt.figure(figsize=(8, 6))
plt.bar(range(kmeans_PCA_heures.n_clusters), variances, color='skyblue')
plt.xlabel('Clusters')
plt.ylabel('Variance intra-classe')
plt.title('Variance intra-classe par cluster')
plt.xticks(range(kmeans_PCA_heures.n_clusters), [f'Cluster {i+1}' for i in range(kmeans_PCA_heures.n_clusters)])
plt.show()


Les variances sont raisonnables, on peut donc continuer nos analyses. 

Y-a-t-il un lien entre 'hill' et les clusters ?

In [ ]:
tbl4_2 = pd.crosstab(clusters_heures_pca, coord['bonus'])

#on affiche la mosaïque :
tbl4_2.index = ['Cluster_1', 'Cluster_2', 'Cluster_3','Cluster_4'] 
tbl4_2.columns = ['Pas colline', 'Colline']  

sns.heatmap(tbl4_2, cmap='coolwarm', annot=True, fmt="d")
plt.title("Stations en altitude en fonction du clusters")
plt.show

Effectivement, les stations en altitudes sont regroupées dans un unique cluster comme pour tous les autres jeux de données, le cluster_3.

On va afficher maintenant le chargement moyen des stations par cluster au cours de la semaine. Cela va peut-être nous permettre de visualiser des tendances communes entre les éléments d'un même cluster.

In [ ]:
loading_sub6 = loading.iloc[:,:]
loading_sub6['cluster']=clusters_heures_pca
mean_loading6=loading_sub6.groupby('cluster').mean()
mean_loading6.head()

fig, ax = plt.subplots(figsize=(10, 6))
col =  ['#FF69B4', '#008080'  ,'#800080','#FFFF00']

# Affichage des 4 lignes sur le même graphique
for i in range(4):
    ax.plot(mean_loading6.columns, mean_loading6.iloc[i], label=f'Cluster {i+1}',color=col[i])

    
plt.legend()
plt.title('Moyenne des chargements par cluster')
plt.xlabel('Temps en heures')
plt.ylabel('Moyenne des loading')
plt.show()

L'interprétation est la même que celle du jeu de données complet.

Comparaison des 2 jeux de données

On compare les deux classifications grâce à un scatterplot pour voir le nombre de valeurs qui correspondent dans le cas normal et le cas réduit, afin de voir si il est vraiment utile de faire son analyse sur l'ACP ou non  :

Tout d'abord on reclasse les clusters pour qu'ils correspondent bien

In [ ]:
 cm3,res_pca_sorted = matchClasses(clusters_heures, clusters_heures_pca)

In [ ]:
Position = np.array(coord)[:,0:2]

#graphe à pour data_jours entier
f1 = px.scatter_mapbox(clusters_heures, Position[:,1], Position[:,0], color=clusters_heures, color_continuous_scale=[(0, '#800080'), (0.5, '#008080'), (1, '#FFFF00')], 
                        size_max=15, zoom=10,mapbox_style="carto-positron", opacity = .9,
                        title = 'clusters données complètes')

f1.show()

#graphe à pour data_jours_pca
f2 = px.scatter_mapbox(res_pca_sorted, Position[:,1], Position[:,0], color=res_pca_sorted, color_continuous_scale =[(0, '#800080'), (0.5, '#008080'), (1, '#FFFF00')], 
                        size_max=15, zoom=10,mapbox_style="carto-positron", opacity = .9,
                        title = 'clusters données pca')
f2.show()



points_diff = clusters_heures != res_pca_sorted

# Calcul du nombre de points différents
num_diff_points = np.sum(points_diff)

# Calcul du pourcentage de réussite
pourcentage_reussite = (1- num_diff_points / len(clusters_heures)) * 100

# Affichage du résultat
print(f"Nombre de points différents : {num_diff_points} sur {len(clusters_heures)}")
print(f"Pourcentage de réussite : {pourcentage_reussite:.2f}%")




Le pourcentage de réussite montre que le clustering sur le jeu de données réduit représente bien le jeu de données complet. On peut donc utiliser le jeu réduit pour la suite des analyses.

On affiche la matrice de confusion pour vérifier ce résultat.

In [ ]:
ConfusionMatrixDisplay(cm3).plot()
plt.xlabel('Kmeans sur les données heures réduites (PCA)')
plt.ylabel('Kmeans sur les données heures')
plt.show()


Le jeu de données complet et le jeu réduit ont des clusters qui correspondent bien, la  matrice est quasi diagonale. La conclusion est la même qu'au dessus.

In [ ]:
#on convertit la table de confusion en dataframe 
df_cm3 = pd.DataFrame(cm3, columns=['groupe0','groupe1', 'groupe2','groupe3'], index=['groupe_pca0', 'groupe_pca1', 'groupe_pca2', 'groupe pca3'])
df_cm3.head()

Ainsi, on vient de montrer que nos classifications Kmeans sur le jeu de données complets et réduits par la PCA formés des clusters similaires. Ce qui va nous permettre de travailler sur les données réduites pour la suite.

#### 5.1.2. CAH : Agglomerative Clustering

Etudions maintenant la méthode de la classification ascendante hiérarchique (CAH). Cette méthode consiste à afficher des dendrogrammes des différentes données en fonction de différents likages.

Comme expliqué précedemment, on préférera faire l'analyse en prenant un échantillon plus petit pour minimiser les temps de calculs. Nous avons vu que l'analyse des données PCA donnait de bons résultats, donc pour faciliter les calculs, lors des analyses suivantes nous prendrons les données réduites

In [ ]:
print("Dimension du jeu de données entier 'loading' :",data_heures.shape)
print("Dimension du jeu de données réduit 'loading_pca': "  +str(data_heures_pca.shape))

On voit bien que la dimension est bien réduite.

On détermine le nombre de clusters adéquats grâce au graphe du coude :

In [ ]:
#Réalisation du CAH avec le linkage 'Ward' 
ac = AgglomerativeClustering(linkage="ward", compute_distances=True)
clusters_cah3 = ac.fit_predict(data_heures_pca)

#Affichage du nombre de clusters à garder
ac = AgglomerativeClustering(linkage='ward', compute_distances=True)
visualizer = KElbowVisualizer(ac, k=(1,12))
visualizer.fit(data_heures_pca)  
visualizer.show()   
plt.show()

D'après les résultats graphiques obtenus en R et en Python, nous décidons de prendre 4 clusters différents.

On affiche alors le dendrogramme associé en utilisant le linkage "Ward".

In [ ]:
#Nombre de clusters choisi par les indicateurs 
K = 4

#Réalisation du CAH avec le linkage 'Ward' & affichage du résultat en coupant à K = 4 clusters
ac = AgglomerativeClustering(n_clusters=K, compute_distances=True, linkage='ward')
clusterscah3 = ac.fit(data_heures_pca)

children = ac.children_
distances = ac.distances_
n_observations = np.arange(2, children.shape[0]+2)
linkage_matrix = np.c_[children, distances, n_observations]

sch.dendrogram(linkage_matrix, labels=ac.labels_)

# On coupe le dendrogramme à K=4 clusters
max_d = .5*(ac.distances_[-K]+ac.distances_[-K+1])
plt.axhline(y=max_d, c='k')

plt.title("Dendrogramme avec le linkage Ward")
plt.show()

On affiche tous les types de linkages que l'on a étudié pour montrer que "Ward" est le type dont le graphe est le plus facile à explorer. 

In [ ]:
#Affichage des dendrogrammes avec les différents linkages afin de pouvoir voir les différences sur un même jeu de données
plt.subplot(2,2,1)
linkage_matrix_single = sch.linkage(data_heures_pca, method='single')
sch.dendrogram(linkage_matrix_single)
plt.title("Dendrogramme avec le linkage simple")

plt.subplot(2,2,2)
linkage_matrix_complete = sch.linkage(data_heures_pca, method='complete')
sch.dendrogram(linkage_matrix_complete)
plt.title("Dendrogramme avec le linkage complete")

plt.subplot(2,2,3)
linkage_matrix_average = sch.linkage(data_heures_pca, method='average')
sch.dendrogram(linkage_matrix_average)
plt.title("Dendrogramme avec le linkage moyenne")

plt.subplot(2,2,4)
linkage_matrix_ward = sch.linkage(data_heures_pca, method='ward')
sch.dendrogram(linkage_matrix_ward)
plt.title("Dendrogramme avec le linkage Ward")

plt.tight_layout()
plt.show()

De plus, on voit bien graphiquement que 3 clusters correspond bien au 'jump' le plus important dans le graphe de 'Ward'. 

Enfin, la mosaïque sur R nous informe quele cluster_3 comporte le plus de stations situées sur des collines mais les autres clusters en contiennent également une bonne quantité donc on ne peut pas faire de liens explicites entre collines et clusters.

#### 5.1.3. Gaussian Mixture Model

On peut utiliser les données issues de l'ACP car elle résume bien le jeu de donnée divisées par les heures

Afin de mettre en place la méthode du GMM, on applique un critère de sélection du nombre de clusters. On choisit ici de travailler avec le crière BIC.

In [ ]:
# Critère BIC :

In [ ]:
k_max = 15

bic = []
for k in range(2, k_max):
    gmm = GaussianMixture(n_components=k, init_params='kmeans', n_init=3)
    gmm.fit(data_heures_pca)
    bic.append(gmm.bic(data_heures_pca))
bic = np.array(bic)

plt.scatter(range(2, k_max), bic)
plt.show()

En appliquant le critère BIC en Python on devrait garder entre 6 et 8 variables. En R on trouve aussi qu'il faudrait garder 8 variables, mais le critère ne varie plus vraiment après 4 clusters. 

Prendre 8 clusters ne conviendrait pas à l'interprétation que l'on cherche à réaliser sur ceux-ci. On décide donc de prendre le risque de garder le même nombre de clusters qu'avec Kmeans c'est à dire 4. 

On affiche d'abord la dispersion des points par cluster initiée par le modèle de mélange gaussien (GMM)

In [ ]:
K = 4
cmap = plt.get_cmap('Set3', K)

gmm = GaussianMixture(n_components=K, n_init=3)
clusters_gmm_heures = gmm.fit_predict(data_heures_pca)

# --- #

plt.subplot(1,2,1)
scatter = plt.scatter(data_heures_pca[:,0], data_heures_pca[:,1], c=clusters_gmm_heures, s=1, linewidths=1, cmap='viridis')

legend_labels = [f'Cluster {i+1}' for i in range(4)]
plt.legend(handles=scatter.legend_elements()[0], labels=legend_labels)
    
plt.grid(True)
plt.tight_layout()
plt.show()

Avant de continuer nos analyses
, on va vérifier que la variance dans chaque cluster ne soit pas trop grande, ainsi les élèments d'un même cluster se comporteraient en moyenne de la même manière. <br> On affiche pour cela les boxplots des données de chaque cluster.

In [ ]:
aux = pd.DataFrame({
    'label': ['Cluster ' + str(label) for label in clusters_gmm_heures],
    'proba': np.max(gmm.predict_proba(data_heures_pca), axis=1)
})
sns.boxplot(x='label', y='proba', data=aux, palette='Set3', showfliers=False)
plt.title('Cluster Probabilities')
plt.show()


Les variances des données dans chaque cluster, représentée par la longueur des boxplots, est relativement faible, on peut continuer nos analyses.

On affiche maintenant le chargement moyen des stations par clusters (issus de GMM) au cours de la semaine. Cela va peut-être nous permettre de visualiser des tendances communes entre les éléments d'un même cluster.

In [ ]:
loading_sub5_1 = loading.iloc[:,:]
loading_sub5_1['cluster']=clusters_gmm_heures
mean_loading5=loading_sub5_1.groupby('cluster').mean()
mean_loading5.head()


fig, ax = plt.subplots(figsize=(10, 6))
col= ['purple', 'blue' ,'green','yellow']

# Affichage des 4 lignes sur le même graphique

for i in range(4):
    ax.plot(mean_loading5.columns, mean_loading5.iloc[i], label=f'Cluster {i+1}',color=col[i])

    
plt.legend()
plt.title('Moyenne des chargements par cluster')
plt.xlabel('Temps en heures')
plt.ylabel('Moyenne des loading')
plt.show()

On affiche désormais le plot mosaïque, qui permet de voir combien de stations de chaque cluster sont en altitude ou non:

In [ ]:
tbl3_3 = pd.crosstab(clusters_gmm_heures, coord['bonus'])

#on affiche la mosaïque :
tbl3_3.index = ['Cluster_1', 'Cluster_2', 'Cluster_3','Cluster_4'] 
tbl3_3.columns = ['Pas colline', 'Colline']  

sns.heatmap(tbl3_3, cmap='coolwarm', annot=True, fmt="d")
plt.title("Stations en altitude en fonction du clusters")
plt.show()


Analyse des divers graphes affichés précédement :

Le graphe des moyennes nous montre que les tendances des courbes sont similaires à celles obtenues lors de l'étude de loading.
On retrouve un cluster qui à tendance à se vider le matin et à se remplir à nouveau le soir. Le second cluster correspond aux stations qui se chargent aussi le matin et se remplissent le soir mais qui auront tendances à rester assez remplies. 
Ensuite, on retrouve un cluster qui se charge la journée et se décharge la nuit. Le 4ème à un comportement similaire mais est plus souvent vide.


La mosaïque nous permet de voit qu'un cluster contient la majorité des stations en altitude, ce qui s'accorde avec notre interprétation du graphe des moyennes, concernant les stations qui sont moins chargées car situées sur des collines.

On affiche le biplot qui affiche les clusters de GMM dans le plan d'ACP en R (seulement car la qualité du graphe en Python n'est vraiment pas satisfaisante). Ce graphe nous permet de voir que les points situés positivement sur l'axe de la composante 1 correspondent aux stations fortement chargées (par rapport à l'analyse de l'ACP faites précédemment (composante 1 ~ chargement des stations)). Les points du cluster situé vers l'origine du graphe correspondent aux stations moyennement chargées. Et les points situés négativement sur l'axe de la composante 1 correspondent aux stations peu chargées. Les trois clusters de la méthode GMM semblent donc correspondre aux niveaux de chargement des stations par rapport à la composante 1.

Comparons les  méthodes de classification Kmeans et GMM : 

In [ ]:
cm3_GMM, clusters_pca_sorted_heures_GMM = matchClasses(clusters_gmm_heures, clusters_heures_pca)


ConfusionMatrixDisplay(cm3_GMM).plot()

plt.xlabel('Gmm sur les données heures réduites')
plt.ylabel('Kmeans sur les données heures réduites (PCA)')
plt.show()

df_kmeans_gmm3 = pd.DataFrame(cm3_GMM, columns=['groupe0', 'groupe1','groupe2', 'groupe3'], index=['groupe_pca0', 'groupe_pca1','groupe_pca2', 'groupe_pca3'])
df_kmeans_gmm3.head()

En affichant la table de contingence, on voit que le nombre de stations mal classifié est très faible, cela nous permet de conclure (directement et non en passant par une CA) que nos méthodes de classification sont comparables.

## 6. MCA sur data_heures

Dans cette partie nous allons effectuer une MCA (Analyse par Correspondances Multiples). Son but est d'établir des correspondances entre des variables catégorielles (donc des variables qualitatives). La MCA va permettre de visualiser les relations entre ces variables pour ensuite créer des groupes sous-jacents.

### 6.1. MCA

Avant d'appliquer la MCA, il faut seuiller nos données, car celles-ci sont continues (data_heures), il nous les faut transformer en données qualitatives pour pouvoir faire notre analyse.

In [ ]:
# Définition des intervalles
intervalles = [0, 0.2, 0.6, 1]

# Noms des seuils:
noms_des_seuils = ['-', '=', '+']

#nouveau data_frame :
loading_quali_heures = data_heures.copy(deep=True)

# Transformation des données continues en catégories avec des noms spécifiques
for colonne in loading_quali_heures.columns :
    loading_quali_heures[colonne] = pd.cut(loading_quali_heures[colonne], bins=intervalles, labels=noms_des_seuils, include_lowest=True)

# Rajout d'une colonne pour signaler l'altitude
loading_quali_heures['Hill'] = coord['bonus'].map({0: 'FAUX', 1: 'VRAI'})

display(loading_quali_heures)

In [ ]:
# Calcul MCA
mca3 = prince.MCA(
    n_components=21,  # Le nombre de composantes voulues pour l'AMC
    n_iter=10,  
    copy=True,  
    check_input=True, 
    engine='sklearn',  
    random_state=42  
)

# Adapter le modèle 
mca3 = mca3.fit(loading_quali_heures)

# Transformer les données
loading_heures_mca = mca3.transform(loading_quali_heures)

# Affichage des résultats
print(loading_heures_mca)

In [ ]:
## inertie portée par chaque composante
display(mca3.eigenvalues_summary)
mca3.scree_plot ()

D'après les résultats précédents, pour expliquer 80% de la variance il faudrait garder 7 composantes de l'AMC. Ce résultat se retrouve ici, dans le graphique suivant où l'on observe un changement de tendance de la courbe entre 6 et 8.

In [ ]:
# Visualisation du graphe du coude
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(mca3.eigenvalues_) + 1), mca3.eigenvalues_, 'o-', color='pink')
plt.title('Scree Plot')
plt.xlabel('Dimensions')
plt.ylabel('Valeurs Propres')
plt.xticks(range(1, len(mca3.eigenvalues_) + 1))
plt.grid(True)
plt.show()

Dans le graphe précédent, par la méthode du coude on peut voir que le nombre de composantes idéal est entre 6 et 9. On choisit pour la suite 7 composantes.

In [ ]:
mca3 = prince.MCA(n_components = 7)
mca3 = mca3.fit(loading_quali_heures)
#On fixe le nombre de composantes à 7

Regardons tout d'abord, si les variables sont bien représentées.

In [ ]:
# Qualité de représentation
mca3.column_cosine_similarities(loading_quali_heures)
#afficher ce qui cos

Ces valeurs vont de 0 (moins bonne représentation) à 1. Pour les composantes 0, 1 et 2 les valeurs du cos2 sont plutot importantes donc on peut dire que oui, les variables sont bien représentées dans le plan d'AMC 0, 1 et 2. Mais on rappelle que ces 3 composantes expliquent seulement 55% de variance.

On visualise ici la répartition résultats de la MCA dans le plan factoriel

In [ ]:
mca3.plot(
    loading_quali_heures,
    x_component=0,
    y_component=1,
    show_column_markers=True,
    show_row_markers=True,
    show_column_labels=True,
    show_row_labels=False
)

On remarque ici que la variable 'Hill_vrai' est situé dans le plan MCA à coté de stations peu chargées en moyenne, elle est négativement corrélées à la dimension 0. Cela rejoint l'interprétation de la variable 'Hill' que nous avons depuis le debut du projet.
De plus, on regarde que la composante 0 semble représenter le chargement moyen. Plus on est corrélé positivement, plus on est chargé en moyenne.

In [ ]:
mca3.plot(
    loading_quali_heures,
    x_component=0,
    y_component=2,
    show_column_markers=True,
    show_row_markers=True,
    show_column_labels=True,
    show_row_labels=False
)

Le graphe suivant va permerttre de visualiser les variables réduites par la MCA avec la variable 'Hill' dans le plan factoriel, on pourra peut etre trouver une tendance particulière.

Quelles variables contribuent le plus aux axes ? Trouve-t-on un comportement général ?

In [ ]:
contrib3 = mca3.column_contributions_.style.format('{:.1%}')
display(contrib3.highlight_max(color='orange').highlight_min(color='lightblue'))

In [ ]:
contrib3 = mca3.column_contributions_.iloc[:, :3]
print("-------contribution > 0.025 pour comp0---------")
contrib3_C0 = contrib3.iloc[:, 0]
C0_filtrée = contrib3_C0[contrib3_C0 > 0.025]
display(C0_filtrée)

print("-------contribution > 0.025 pour comp1---------")
contrib3_C1 = contrib3.iloc[:, 1]
C1_filtrée = contrib3_C1[contrib3_C1 > 0.025]
display(C1_filtrée)

print("-------contribution > 0.025 pour comp2---------")
contrib3_C2 = contrib3.iloc[:, 2]
C2_filtrée = contrib3_C2[contrib3_C2 > 0.025]
display(C2_filtrée)

On peut voir que pour la composante 0, c'est les heures de nuit chargées qui contribuent fortement. Pour la composante 1, plutôt les heures de journée chargées et celles de nuit faiblement chargées et pour la composante 2 les heures de nuits moyennement chargées.

### 6.2. Méthode de clustering K-means

Une fois l'analyse MCA faite, on va tenter de trouver des liens entre les variables, des comportements communs. Pour cela on effectue du clustering et plus précisement la méthode K-Means.

On cherche tout d'abord le nombre de clusters optimal.

In [ ]:
inertia = []
for k in range(1, 15):
    kmeans_mca3 = KMeans(n_clusters=k, init='random', n_init='auto', max_iter=100, random_state=42)
    kmeans_mca3.fit(loading_heures_mca)
    inertia.append(kmeans_mca3.inertia_)

inertia = np.array(inertia)


plt.scatter(range(2, 15), inertia[1:])
plt.xlabel('nombre de clusters', fontsize = 15)
plt.ylabel('inertie', fontsize = 15)
plt.title("Graphe du coude", fontsize = 15)
plt.show()


# --- #


# Using yellowbrick

kmeans_mca3 = KMeans(init='random', n_init='auto', max_iter=100, random_state=42)
visualizer = KElbowVisualizer(kmeans_mca3, k=(1,12))

visualizer.fit(loading_heures_mca)    # Fit the data to the visualizer
visualizer.show()    # Finalize and render the figure

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15,8))

for k in range(2, 6):
    kmeans_mca3 = KMeans(n_clusters=k, init='random', n_init='auto', max_iter=100, random_state=42)
    q, mod = divmod(k, 2) 
    # Create SilhouetteVisualizer instance with KMeans instance
    visualizer = SilhouetteVisualizer(kmeans_mca3, colors='yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(loading_heures_mca)

Comme pour le clustering K-means sur les données PCA, on trouve ici que le nombre idéal de clusters est 3 ou 4. On penche plus vers 4 pour voir si l'analyse avec PCA ou avec MCA est comparable.

In [ ]:
#on fixe le nombre de clusters à 4
kmeans_mca3 = KMeans(n_clusters=4, init='random', n_init='auto', max_iter=100, random_state=42)
clusters_mca3 = kmeans_mca3.fit_predict(loading_heures_mca)

Avant de continuer nos analyses, nous allons calculer la variance intra-classe de chaque cluster afin de s'assurer que les éléments d'un même cluster ont un comportement similaire.

In [ ]:
# distance de chaque point de données à son centre de cluster
distances = kmeans_mca3.transform(loading_heures_mca)

#variance intraclasse pour chaque cluster
variances = []
for cluster in range(kmeans_mca3.n_clusters):
    cluster_distances = distances[kmeans_mca3.labels_ == cluster, cluster]
    variance = np.var(cluster_distances)
    variances.append(variance)

# Affichage
plt.figure(figsize=(8, 6))
plt.bar(range(kmeans_mca3.n_clusters), variances, color='skyblue')
plt.xlabel('Clusters')
plt.ylabel('Variance intra-classe')
plt.title('Variance intra-classe par cluster')
plt.xticks(range(kmeans_mca3.n_clusters), [f'Cluster {i+1}' for i in range(kmeans_mca3.n_clusters)])
plt.show()


Les variances intra-classes de chaque clusters sont très faibles donc on peut poursuivre nos analyses.

Le cluster 2 à une variance assez élevée comparée à celles des autres clusters qui sont plutot faibles.

In [ ]:
clusters_mca3 = kmeans_mca3.fit_predict(loading_heures_mca)
loading_sub_mca3 = loading.iloc[:,:]
loading_sub_mca3['cluster']=clusters_mca3
mean_loading_mca3=loading_sub_mca3.groupby('cluster').mean()
mean_loading_mca3.head()

fig, ax = plt.subplots(figsize=(10, 6))

col= ['purple', 'blue'  ,'green','#FFFF00']

# Affichage des 4 lignes sur le même graphique
for i in range(4):
    ax.plot(mean_loading_mca3.columns, mean_loading_mca3.iloc[i], label=f'Cluster {i+1}', color=col[i])

plt.legend()
plt.title('Moyenne des chargements par cluster')
plt.xlabel('Temps en heures')
plt.ylabel('Moyenne des loading')
plt.show()


L'évolution du chargement moyen des stations sur la semaine semble avoir le même comportement que pour le jeu de données regroupé par heure.

Ci dessous on peut visualiser ces clusters sur la carte de Paris.

In [ ]:
f1 = px.scatter_mapbox(clusters_mca3, Position[:,1], Position[:,0], color=clusters_mca3, color_continuous_scale=[(0, '#800080'), (0.5, '#008080'), (1, '#FFFF00')], 
                        size_max=15, zoom=10,mapbox_style="carto-positron", opacity = .9,
                        title = 'clusters données complètes')

f1.show()

### 6.3. Gaussian Mixture Model

On effectue une deuxième méthode de clustering sur la mca précedente pour voir si certains résultats sont intéressants.

Le graphe suivant nous permet de selectionner le nombre de clusters optimal avec la méthode GMM.

In [ ]:
k_max = 15

bic = []
for k in range(2, k_max):
    gmm = GaussianMixture(n_components=k, init_params='kmeans', n_init=3)
    gmm.fit(loading_heures_mca)
    bic.append(gmm.bic(loading_heures_mca))
bic = np.array(bic)

plt.scatter(range(2, k_max), bic)
plt.show()

Avec cette méthode on devrait garder entre 6 et 8 clusters. Pour la suite on garde seulement 4 clusters car avec 7-8 l'interprétation serait trop compliquée.

In [ ]:
K = 4
cmap = plt.get_cmap('Set3', K)

gmm = GaussianMixture(n_components=K, n_init=4)
clusters_gmm_heures_mca = gmm.fit_predict(loading_heures_mca)
print(clusters_gmm_heures_mca)
# --- #

plt.subplot(1,2,1)
plt.scatter(loading_heures_mca.iloc[:,0], loading_heures_mca.iloc[:,1], c=clusters_gmm_heures_mca, s=1, linewidths=1, cmap='viridis')
plt.grid(True)


plt.tight_layout()
plt.show()

Avant de continuer nos analyses
, on va vérifier que la variance dans chaque cluster ne soit pas trop grande, ainsi les élèments d'un même cluster se comporteraient en moyenne de la même manière. <br> On affiche pour cela les boxplots des données de chaque cluster.

In [ ]:
aux = pd.DataFrame({
    'label': ['Cluster ' + str(label) for label in clusters_gmm_heures_mca],
    'proba': np.max(gmm.predict_proba(loading_heures_mca), axis=1)
})
sns.boxplot(x='label', y='proba', data=aux, palette='Set3', showfliers=False)
plt.title('Cluster Probabilities')
plt.show()


Les variances des données dans chaque cluster, représentée par la longueur des boxplots, est très faible pour les 4 clusters, on peut poursuivre nos analyses.

In [ ]:
loading_sub_mca3 = loading.iloc[:,:]
loading_sub_mca3['cluster']=clusters_gmm_heures_mca
mean_loading_mca3=loading_sub_mca3.groupby('cluster').mean()
mean_loading_mca3.head()

fig, ax = plt.subplots(figsize=(10, 6))

col= ['purple', 'blue'  ,'green','#FFFF00']

# Affichage des 4 lignes sur le même graphique
for i in range(4):
    ax.plot(mean_loading_mca3.columns, mean_loading_mca3.iloc[i], label=f'Cluster {i+1}', color=col[i])

plt.legend()
plt.title('Moyenne des chargements par cluster')
plt.xlabel('Temps en heures')
plt.ylabel('Moyenne des loading')
plt.show()


In [ ]:
f1 = px.scatter_mapbox(clusters_gmm_heures_mca, Position[:,1], Position[:,0], color=clusters_gmm_heures_mca, color_continuous_scale=[(0, '#800080'), (0.5, '#008080'), (1, '#FFFF00')], 
                        size_max=15, zoom=10,mapbox_style="carto-positron", opacity = .9,
                        title = 'clusters données complètes')

f1.show()

Avec GMM sur la MCA, on retrouve à peu près les mêmes conclusions qu'avec les k-means/cah sur loading et df1. Le cluster 4 correspond aux stations de travail chargées le jour et vides la nuit. Le cluster 1 est l'inverse du cluster 4 et le cluster 3 correspond aux stations dont le chargement varie peu. Les clusters 1 et 2 pourraient correspondre à des zones résidentielles mais également fréquentés le week-end donc aussi à des zones de divertissement alors que le cluster 3 le chargement varie beaucoup moins le week-end donc beaucoup moins fréquenté.

On crée une fonction qui prédit à quel cluster pourrait appartenir une nouvelle station implementée à une latitude x et une longitude y. On regarde la station la plus proche de celle-ci en terme de coordonnées et on lui attribue le même cluster que cette station. \
Dans notre exemple, la station avec une latitude 2.35 et une longitude 48.45 appartiendrait au cluster 2, ie au cluster des stations qui se chargent en journée qui correspond aux stations proche des lieux de travails. 

In [ ]:
def def_pred_station(coord1, coord2, coord, reskmeans):
    min_distance = 1000
    r = 1
    for i in range(coord.shape[0]):
        resultat = np.sqrt((coord1 - coord.iloc[i, 0])**2 + (coord2 - coord.iloc[i, 1])**2)
        if resultat < min_distance:
            min_distance = resultat
            r = i
    
    result = reskmeans[r]
    localisation = coord.iloc[r]
    return result, localisation


In [ ]:
def_pred_station(2.35 , 48.45, coord, clusters)

## 7. Conclusion

Dans ce projet, nous avons remarqué que le profil de chargement d'une station dépend de sa localisation. Le chargement varie surtout en fonction des heures de la journée mais un peu moins entre le week-end et la semaine. \
Dans nos méthodes de clustering K-Means et CAH, on a repéré 4 types de stations : 
- Ls stations qui se chargent en journée et qui se vident en soirée, qui correspondent aux zones de travail. Ces stations sont également en moyenne moins chargées et un peu moins active le week-end. 

- Les stations qui se chargent en soirée et se vide le matin : elles correspondent aux zones d'habitation. Leur chargement reste élevé le week-end et varie moins car les gens n'ont pas besoin de se rendre à leur lieu de travail. 
- Les stations avec un chargement élevé en moyenne mais dont le chargement ne varie pas beaucoup 
- les stations avec un chargement faible en moyenne et dont le chargement ne varie pas beaucoup. On a vu que les stations en altitude sont en majorité dans ce dernier cluster. 

Ces deux derniers types de clusters correspondent aux stations moins empruntées. 

Le GMM dépend du langage de programmation : en R il nous rend en moyenne 2/3 clusters : il différencie les stations par leur chargement moyen et non pas par la variabilité de leur chargement. En Python, il renvoie les mêmes clusters que CAH/K-Means.